In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
# print(os.getcwd())
# os.chdir('./GAN-Estimation-Uncertainty/uncertainty')

In [ ]:
import numpy as np

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils import data
import torchvision
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import math
from tqdm import tqdm

In [ ]:
# import seaborn as sea

### Useful sites:
1. [Torch-Uncertainty Library](https://github.com/ensta-u2is/torch-uncertainty)
   - A library for uncertainty estimation in deep learning, including deep learning baselines like Deep Ensembles, MC-Dropout, and more. The repository also provides tutorials.

2. [Awesome Uncertainty Deep Learning Repository](https://github.com/ENSTA-U2IS/awesome-uncertainty-deeplearning)
   - A collection of papers and resources related to Uncertainty and Deep Learning.

3. [Paper on Uncertainty Estimation](https://cds.cern.ch/record/2837844/files/2210.09767.pdf)
   - A paper from HSE describing how uncertainty estimation was applied to a GAN task in a physics experiment, specifically the LHCb experiment.


### Prepare test dataset

In [ ]:
from datasets.sine import SinDataset

In [ ]:
my_dataset = SinDataset(
                  train_data_length = 1024,
                  train_data_length_certain = 974,
                  multiplier_2 = 3
    
                )

# my_dataset = SinDataset(
#                   train_data_length = 990,
#                   train_data_length_certain = 990
#                 )


In [ ]:
# my_dataset[:][1]

In [ ]:
plt.plot(my_dataset[:][1], my_dataset[:][0], 'ko', markersize=0.5)

save_filename = os.path.join('datasets', 'sine')
plt.savefig(save_filename, dpi=300)

In [ ]:
BATCH_SIZE = 32
train_loader = torch.utils.data.DataLoader(
    my_dataset, batch_size=BATCH_SIZE, shuffle=True
)

### GAN architecture
>start with basic simple cGAN

Define the GAN

>The GAN is designed such that a random latent number or vector is concatenated with a given variable 'x.' The GAN is then trained to learn the mapping from this combined input to the target variable 'y,' where 'y' is defined as the sine of 'x'

In [ ]:
from modules.base_GAN import Base_Discriminator, Base_Generator, train
from modules.optimal_batch import WeightedIntervalCrossEntropyLoss
from modules.optimal_batch import create_batch, calculate_variance, weights_variances
from modules.animate import plot_sine
from modules.optimal_batch import calculate_MSE, calculate_metrics

In [ ]:
from modules.optimal_batch import WeightedVarianceCrossEntropyLoss, variance4data

In [ ]:
loss_f = WeightedVarianceCrossEntropyLoss()

In [ ]:
from modules.mnist_models import CNNClassifierWrapper

In [ ]:
# from modules.mnist_classifier import eval_model
# from modules.mnist_models import CNN, CNN2, CNN3
# CLASSIFIER = CNN3().to(device=device)
# PATH = './mnist_tests_classifier/classifier__CNN3__1_9_32/classifier_model.pt'
# CLASSIFIER.load_state_dict(torch.load(PATH))
D.eval()

In [ ]:
class DWrapper:
    def __init__(self, model, layer_index=-1, use_global_pooling=False):
        self.model = model
        self.activation = {}
        self.base_transform = torchvision.transforms.Compose([
                            torchvision.transforms.ToTensor(),
                            torchvision.transforms.Normalize(mean=(0.5,), std=(0.5,))
                        ])
        self.layer_index = layer_index
        self.model.model[self.layer_index].register_forward_hook(self.get_activation('extract'))
        self.use_global_pooling = use_global_pooling
        
    def get_activation(self, name):
        def hook(model, input, output):
            self.activation[name] = output.detach()
        return hook
    
    def preprocess_image(self, image):
        # Apply transformations to the image
        preprocessed_image = self.base_transform(image)
        # .unsqueeze(0)  # Add batch dimension
        return preprocessed_image

    def __call__(self, image, info, transform = False):
        
        # Preprocess the image
        if transform:
            preprocessed_image = self.preprocess_image(image)
        output = self.model(image, info)
        output = self.activation['extract']
        
        if self.use_global_pooling:
            output = F.adaptive_avg_pool2d(output, (1, 1))

        output = output.view(output.size(0), -1)  # Flatten to (batch_size, num_channels)
        
        return output

In [ ]:
def nvariance4data(generator,
                   discriminator_wrapper,
                   data_loader,
                   repeat: int = 20,
                   device='cpu'):
    variances = {}
    latent_space_samples = torch.randn(repeat, 1)
    for batch_idx, (x, info) in enumerate(data_loader):
        batch_size = x.size(0)
        x, info = x.to(device), info.to(device)
        for el in info:
            with torch.no_grad():
                res = generator(latent_space_samples, el.repeat(repeat, 1))
                # print()
                res = discriminator_wrapper(res, el.repeat(repeat, 1))
                variance = torch.var(res, dim=0)
                variances[el.item()] = torch.sum(variance).item()
                
                
    def variances2weights(variances: dict):
        """
        Convert dict of variances to dict of weights,
        so, the max weight is 1 for max value of variances and 0 if variance is 0
        """
        max_variance = max(variances.values())
        weights = {key: max(0.01, value / max_variance) for key, value in variances.items()} 
        # print(weights)
        return weights


    return variances2weights(variances)

In [ ]:
# plt.bar(nvariance4data(G, FEATURE_EXTRACTOR, train_loader))

In [ ]:
# FEATURE_EXTRACTOR = CNNClassifierWrapper(D, layer_index = -3, use_global_pooling = False)
FEATURE_EXTRACTOR = DWrapper(D, layer_index = -5, use_global_pooling = False)

In [ ]:
# img_example = torch.rand(32, 1, 28, 28)  #image 28 on 28 with 1 chanel like in mnist
for a, b in train_loader:
    break
    
z = torch.randn(a.size(0), 1)
x_fake, y_fake = G(z, b.view(-1, 1)).view(-1, 1), torch.zeros(a.size(0), 1) 
print(x_fake.size(), b.view(-1, 1).size())
print(D(x_fake, b.view(-1, 1)).size())
print(FEATURE_EXTRACTOR(x_fake, b.view(-1, 1)).size())

In [ ]:
# x_fake
# y_fake

In [ ]:
# D.eval
# D(b)
# D(x_fake, b.view(-1, 1))


In [ ]:
for a, b in train_loader:
    break

In [ ]:
# a = variance4data(G, train_loader)
# for batch_idx, (x, info) in enumerate(train_loader):
#     batch_size = x.size(0)
#     x, info = x.to(device), info.to(device)
#     break
    
# G.zero_grad()

# z = torch.randn(x.size(0), 1).to(device)
# y = torch.ones(x.size(0), 1).to(device)
# G_output = G(z, info.view(-1, 1)).view(-1, 1)
    
# D_output = D(G_output, info.view(-1, 1))

# G_loss = loss_f(D_output, y.to(device), weights = a, conditional_info = info.view(-1, 1))
# G_loss

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)

In [ ]:
#model parametrs
# learning_rate_G = 0.0001
# learning_rate_D = 0.0004
# lr = 0.001
lr = 0.0001
NUM_EPOCHS = 1000
NUM_BATCHES = math.ceil(len(train_loader.dataset)/train_loader.batch_size)
print(f'NUM_BATCHES: {NUM_BATCHES}')

In [ ]:
NUM_BATCHES = math.ceil(len(train_loader.dataset)/train_loader.batch_size)
print(f'NUM_BATCHES: {NUM_BATCHES}')

In [ ]:
# loss_function = nn.BCELoss()
loss_function = WeightedIntervalCrossEntropyLoss()

In [ ]:
D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# D_optimizer = optim.Adam(D.parameters(), lr=learning_rate_G, betas=(0.5, 0.999))
# G_optimizer = optim.Adam(G.parameters(), lr=learning_rate_D, betas=(0.5, 0.999))

from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import OneCycleLR

max_lr = 0.0005

scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)

# scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 8001)
# scheduler_D = CosineAnnealingWarmRestarts(D_optimizer, T_0 = 16001)
# scheduler_G = CosineAnnealingWarmRestarts(G_optimizer, T_0 = 16001)

In [ ]:
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
EPOCHS = 1000
BATCHES = 32
steps = []
lrs = []
model = Base_Discriminator().to(device=device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 960)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 16001)
for epoch in range(EPOCHS):
    for batch in range(BATCHES):
        scheduler.step()
        lrs.append(scheduler.get_last_lr()[0])
        steps.append(epoch * BATCHES + batch)

plt.figure()
plt.legend()
plt.plot(steps, lrs, label='OneCycle')
plt.show()

In [ ]:
EPOCHS = 10
BATCHES = 10
steps = []
lrs = []
model = Base_Discriminator().to(device=device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=10,total_steps=EPOCHS * BATCHES)
for epoch in range(EPOCHS):
    for batch in range(BATCHES):
        scheduler.step()
        lrs.append(scheduler.get_last_lr()[0])
        steps.append(epoch * BATCHES + batch)

plt.figure()
plt.legend()
plt.plot(steps, lrs, label='OneCycle')
plt.show()

In [ ]:
# lr_scheduler = get_cosine_schedule_with_warmup(optimizer,
#                                                num_warmup_steps=num_warmup_steps,
#                                                num_training_steps=len(train_dataloader) * num_epochs)

In [ ]:
calculate_metrics(G, D)

In [ ]:
# G.eval
torch.tensor(torch.linspace(0, math.pi, 10)[3].item())

In [ ]:
# calculate_MSE

In [ ]:
calculate_MSE(G)


In [ ]:
calculate_MSE(G)

### Training

In [ ]:
# NAME = 'base_gan__variance__2_5'
# NAME = 'base_gan__1_1'
# NAME = 'base_gan__1_1'
# NAME = 'base_gan__variance_10_split__any_wieights__2_1'
# NAME = 'base_gan__variance_2_split__accurate_wieights__3_1'
# NAME = 'base_gan__sin_confidence_all__variance_10_split__accurate_wieights__8_1'
# NAME = 'base_gan__sin_confidence_all_1,5_pi__variance_10_split__accurate_wieights__9_1'
# NAME = 'base_gan__lr_0,0001__10_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights_0,01__lr_0,0001__12_1'
# NAME = 'base_gan__part_dataset__lr_0,0001__9_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights_0,01_OneCycleLR__lr_0,0005__1_1'

save_path = '/Users/serafim/Desktop/Job/projects/science/hse/GAN-Estimation-Uncertainty/uncertainty/tests/'

In [ ]:
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import OneCycleLR
-np.log(1)
# # %matplotlib inline
# %config InlineBackend.close_figures = False

In [ ]:
# torch.var()
a = np.array([[1, 1, 1, 2, 2, 2]])
np.var(a)

In [ ]:
# NUM_EPOCHS
from modules.base_GAN import DWrapper, nvariance4data

In [ ]:
FEATURE_EXTRACTOR = DWrapper(G, layer_index=-2, use_global_pooling=False)
FEATURE_EXTRACTOR.register_hook()  # Register the hook

In [ ]:
# b

In [ ]:
for a, b in train_loader:
    break
z = torch.randn(a.size(0), 1).to(device)
print(z.size())
# y = torch.ones(x.size(0), 1).to(device)
#     # print(z.size(), y.size())
    
#     # fake_info = 2 * math.pi * torch.rand(x.size(0)).view(-1, 1).to(device)#fake info maybe is crooectly to take from data or from the same distribution, with the same density
# G_output = G(z, info.view(-1, 1)).view(-1, space_dimension)

    # weights = nvariance4data(G, FEATURE_EXTRACTOR, data_loader)
    # FEATURE_EXTRACTOR.remove_hook()  
FEATURE_EXTRACTOR(z, b.view(-1, 1)).size()

In [ ]:
a

In [ ]:
# import time
# ts = []

# for i in tqdm(range(1000)):
#     t1 = time.time()
    
#     # FEATURE_EXTRACTOR.to(device)
#     FEATURE_EXTRACTOR = DWrapper(D, layer_index = -5, use_global_pooling = False)
#     weights = nvariance4data(G, FEATURE_EXTRACTOR, train_loader, device = 'cpu')
#     torch.cuda.empty_cache()
#     ts.append(time.time() - t1)
#     FEATURE_EXTRACTOR.activation = {}
#     # print(time.time() - t1)
    
ts = []
for i in tqdm(range(300)):
    t1 = time.time()
    FEATURE_EXTRACTOR = DWrapper(D, layer_index=-5, use_global_pooling=False)
    FEATURE_EXTRACTOR.register_hook()  # Register the hook
    weights = nvariance4data(G, FEATURE_EXTRACTOR, train_loader)
    FEATURE_EXTRACTOR.remove_hook()  # Remove the hook when done
    ts.append(time.time() - t1)
    

In [ ]:
plt.plot(ts)

In [ ]:
# plt.bar(ts)

plt.plot(ts)

In [ ]:
lr = 0.0001
NUM_EPOCHS = 5000

loss_function = WeightedIntervalCrossEntropyLoss()
D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)

D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)

NAME = f'base_gan__variance__10_split__accurate_wieights_0,01_OneCycleLR__lr_0,0001__2'
# Adjust these parameters as needed
max_lr = 0.0005 
scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)


In [ ]:
lr = 0.0001
NUM_EPOCHS = 5000

loss_function = WeightedVarianceCrossEntropyLoss()
# loss_function = WeightedIntervalCrossEntropyLoss()
D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)

D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)

NAME = f'base_gan__variance__each_point__accurate_wieights_0,01_OneCycleLR__lr_0,0001__6'
# Adjust these parameters as needed
max_lr = 0.0005 
scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)


In [ ]:
D_losses_final, G_losses_final, Variances, Weights_variance, s = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = True,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    n_split = 10,
    scheduler_D = scheduler_D,
    scheduler_G = scheduler_G,
    info_n = 200,
)
# plot_sine(G, save_path = save_path, name = NAME)


In [ ]:
D_losses_final, G_losses_final, Variances, Weights_variance, s = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = True,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    n_split = 10,
    scheduler_D = scheduler_D,
    scheduler_G = scheduler_G,
    info_n = 200,
)
# plot_sine(G, save_path = save_path, name = NAME)


In [ ]:
import matplotlib.pyplot as plt

# Define a function to plot the metrics
def plot_metrics(metrics: List[Tuple[float, float, float, float, float, float]], title: str):
    epochs = range(1, len(metrics) + 1)
    
    mse1_values = [m[0] for m in metrics]
    mse2_values = [m[1] for m in metrics]
    var1_values = [m[2] for m in metrics]
    var2_values = [m[3] for m in metrics]
    entropy1_values = [m[4] for m in metrics]
    entropy2_values = [m[5] for m in metrics]
    
    # Plot the metrics
    plt.figure(figsize=(12, 8))
    
    plt.subplot(3, 2, 1)
    # plt.gca().set_ylim([0, 0.1])
    plt.plot(epochs, mse1_values, 'b', label='MSE1')
    plt.title('MSE1')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
    # plt.subplot(3, 2, 2)
    plt.plot(epochs, mse2_values, 'r', label='MSE2')
    plt.title('MSE2')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
    plt.subplot(3, 2, 3)
    # plt.gca().set_ylim([0, 0.1])
    plt.plot(epochs, var1_values, 'g', label='Var1')
    plt.title('Var1')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
    # plt.subplot(3, 2, 4)
    plt.plot(epochs, var2_values, 'c', label='Var2')
    plt.title('Var2')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
    plt.subplot(3, 2, 5)
    plt.plot(epochs, entropy1_values, 'm', label='Entropy1')
    plt.title('Entropy1')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
    # plt.subplot(3, 2, 6)
    plt.plot(epochs, entropy2_values, 'y', label='Entropy2')
    plt.title('Entropy2')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
    plt.tight_layout()
    plt.suptitle(title)
    plt.show()

# Example usage
plot_metrics(s, 'Metrics Over Training')


In [ ]:
import matplotlib.pyplot as plt

# Define a function to plot the metrics
def plot_metrics(metrics: List[Tuple[float, float, float, float, float, float]], title: str):
    epochs = range(1, len(metrics) + 1)
    
    mse1_values = [m[0] for m in metrics]
    mse2_values = [m[1] for m in metrics]
    var1_values = [m[2] for m in metrics]
    var2_values = [m[3] for m in metrics]
    entropy1_values = [m[4] for m in metrics]
    entropy2_values = [m[5] for m in metrics]
    
    # Plot the metrics
    plt.figure(figsize=(12, 8))
    
    plt.subplot(3, 2, 1)
    # plt.gca().set_ylim([0, 0.1])
    plt.plot(epochs, mse1_values, 'b', label='MSE1')
    plt.title('MSE1')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
    # plt.subplot(3, 2, 2)
    plt.plot(epochs, mse2_values, 'r', label='MSE2')
    plt.title('MSE2')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
    plt.subplot(3, 2, 3)
    # plt.gca().set_ylim([0, 0.1])
    plt.plot(epochs, var1_values, 'g', label='Var1')
    plt.title('Var1')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
    # plt.subplot(3, 2, 4)
    plt.plot(epochs, var2_values, 'c', label='Var2')
    plt.title('Var2')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
    plt.subplot(3, 2, 5)
    plt.plot(epochs, entropy1_values, 'm', label='Entropy1')
    plt.title('Entropy1')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
    # plt.subplot(3, 2, 6)
    plt.plot(epochs, entropy2_values, 'y', label='Entropy2')
    plt.title('Entropy2')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
    plt.tight_layout()
    plt.suptitle(title)
    plt.show()

# Example usage
plot_metrics(s, 'Metrics Over Training')


In [ ]:
with open('/Users/serafim/Desktop/Job/projects/science/hse/GAN-Estimation-Uncertainty/uncertainty/tests/base_gan__variance__each_point__accurate_wieights_0,01_OneCycleLR__lr_0,0001__2/metrics.pkl', 'wb') as f:
    pickle.dump(s, f)

In [ ]:
# s

In [ ]:
lr = 0.0001
NUM_EPOCHS = 5000
loss_function = nn.BCELoss()
max_lr = 0.0005

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)

D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
NAME = f'base_gan__lr_0,001_OneCycleLR__7'

scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)

D_losses_final, G_losses_final, Variances, Weights_variance, ss = train(
        num_epochs = NUM_EPOCHS,
        data_loader = train_loader,
        D = D,
        G = G,
        D_optimizer = D_optimizer,
        G_optimizer = G_optimizer, 
        criterion = loss_function,
        device = device,
        plot_process = True,
        save_path = save_path,
        name = f'{NAME}',
        weights_interval = False,
        # plot_info = True,
        animate_bar_var = True,
        progress_generator = True,
        scheduler_D = scheduler_D,
        scheduler_G = scheduler_G,
        info_n = NUM_EPOCHS,
        # save_mse = 'G_mse.pkl'
        # n_split = 10
    )

In [ ]:
lr = 0.0001
NUM_EPOCHS = 5000
loss_function = nn.BCELoss()
max_lr = 0.0005

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)

D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
NAME = f'base_gan__lr_0,001_OneCycleLR__8'

scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)

D_losses_final, G_losses_final, Variances, Weights_variance, ss = train(
        num_epochs = NUM_EPOCHS,
        data_loader = train_loader,
        D = D,
        G = G,
        D_optimizer = D_optimizer,
        G_optimizer = G_optimizer, 
        criterion = loss_function,
        device = device,
        plot_process = True,
        save_path = save_path,
        name = f'{NAME}',
        weights_interval = False,
        # plot_info = True,
        animate_bar_var = True,
        progress_generator = True,
        scheduler_D = scheduler_D,
        scheduler_G = scheduler_G,
        info_n = NUM_EPOCHS,
        # save_mse = 'G_mse.pkl'
        # n_split = 10
    )

In [ ]:
lr = 0.0001
NUM_EPOCHS = 5000
loss_function = nn.BCELoss()
max_lr = 0.0005

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)

D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
NAME = f'base_gan__lr_0,001_OneCycleLR__9'

scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)

D_losses_final, G_losses_final, Variances, Weights_variance, ss = train(
        num_epochs = NUM_EPOCHS,
        data_loader = train_loader,
        D = D,
        G = G,
        D_optimizer = D_optimizer,
        G_optimizer = G_optimizer, 
        criterion = loss_function,
        device = device,
        plot_process = True,
        save_path = save_path,
        name = f'{NAME}',
        weights_interval = False,
        # plot_info = True,
        animate_bar_var = True,
        progress_generator = True,
        scheduler_D = scheduler_D,
        scheduler_G = scheduler_G,
        info_n = NUM_EPOCHS,
        # save_mse = 'G_mse.pkl'
        # n_split = 10
    )

In [ ]:
lr = 0.0001
NUM_EPOCHS = 5000
loss_function = nn.BCELoss()
max_lr = 0.0005

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)

D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
NAME = f'base_gan__lr_0,001_OneCycleLR__10'

scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)

D_losses_final, G_losses_final, Variances, Weights_variance, ss = train(
        num_epochs = NUM_EPOCHS,
        data_loader = train_loader,
        D = D,
        G = G,
        D_optimizer = D_optimizer,
        G_optimizer = G_optimizer, 
        criterion = loss_function,
        device = device,
        plot_process = True,
        save_path = save_path,
        name = f'{NAME}',
        weights_interval = False,
        # plot_info = True,
        animate_bar_var = True,
        progress_generator = True,
        scheduler_D = scheduler_D,
        scheduler_G = scheduler_G,
        info_n = NUM_EPOCHS,
        # save_mse = 'G_mse.pkl'
        # n_split = 10
    )

In [ ]:
# ss

In [ ]:
import matplotlib.pyplot as plt

# Define a function to plot the metrics
def plot_metrics(metrics: List[Tuple[float, float, float, float, float, float]], title: str):
    epochs = range(1, len(metrics) + 1)
    
    mse1_values = [m[0] for m in metrics]
    mse2_values = [m[1] for m in metrics]
    var1_values = [m[2] for m in metrics]
    var2_values = [m[3] for m in metrics]
    entropy1_values = [m[4] for m in metrics]
    entropy2_values = [m[5] for m in metrics]
    
    # Plot the metrics
    plt.figure(figsize=(12, 8))
    
    plt.subplot(3, 2, 1)
    # plt.gca().set_ylim([0, 0.1])
    plt.plot(epochs, mse1_values, 'b', label='MSE1')
    plt.title('MSE1')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
    # plt.subplot(3, 2, 2)
    plt.plot(epochs, mse2_values, 'r', label='MSE2')
    plt.title('MSE2')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
    plt.subplot(3, 2, 3)
    plt.gca().set_ylim([0, 0.1])
    plt.plot(epochs, var1_values, 'g', label='Var1')
    plt.title('Var1')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
    # plt.subplot(3, 2, 4)
    plt.plot(epochs, var2_values, 'c', label='Var2')
    plt.title('Var2')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
    plt.subplot(3, 2, 5)
    plt.plot(epochs, entropy1_values, 'm', label='Entropy1')
    plt.title('Entropy1')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
    # plt.subplot(3, 2, 6)
    plt.plot(epochs, entropy2_values, 'y', label='Entropy2')
    plt.title('Entropy2')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
    plt.tight_layout()
    plt.suptitle(title)
    plt.show()

# Example usage
plot_metrics(s, 'Metrics Over Training')


In [ ]:
with open('/Users/serafim/Desktop/Job/projects/science/hse/GAN-Estimation-Uncertainty/uncertainty/tests/base_gan__lr_0,001_OneCycleLR__2/metrics.pkl', 'wb') as f:
    pickle.dump(s, f)

In [ ]:
with open('/Users/serafim/Desktop/Job/projects/science/hse/GAN-Estimation-Uncertainty/uncertainty/tests/base_gan__variance__10_split__accurate_wieights_0,01_OneCycleLR__lr_0,0001__2/metrics.pkl', 'wb') as f:
    pickle.dump(s, f)

In [ ]:
# import pickle
# with open('/Users/serafim/Desktop/Job/projects/science/hse/GAN-Estimation-Uncertainty/uncertainty/tests/base_gan__lr_0,001_OneCycleLR__2/metrics.pkl', 'rb') as f:
#     ss = pickle.load(f)
    
# with open('/Users/serafim/Desktop/Job/projects/science/hse/GAN-Estimation-Uncertainty/uncertainty/tests/base_gan__lr_0,001_OneCycleLR__2/metrics.pkl', 'rb') as f:
    # mynewlist = pickle.load(f)
    
filename = '/Users/serafim/Desktop/Job/projects/science/hse/GAN-Estimation-Uncertainty/uncertainty/tests/base_gan__lr_0,001_OneCycleLR__2/metrics.pkl'
filename = '/Users/serafim/Desktop/Job/projects/science/hse/GAN-Estimation-Uncertainty/uncertainty/tests/base_gan__variance__10_split__accurate_wieights_0,01_OneCycleLR__lr_0,0001__2/metrics.pkl'
filename = '/Users/serafim/Desktop/Job/projects/science/hse/GAN-Estimation-Uncertainty/uncertainty/tests/base_gan__lr_0,001_OneCycleLR__1/metrics.pkl'
with open(filename, 'rb') as f:
    classification_dict = pickle.load(f)

In [ ]:
# plot_metrics(ss, 'Variation')
import matplotlib.pyplot as plt
from typing import List, Tuple
# Define a function to plot the metrics
def plot_metrics(metrics: List[Tuple[float, float, float, float, float, float]], title: str):
    epochs = range(1, len(metrics) + 1)
    
    mse1_values = [m[0] for m in metrics]
    mse2_values = [m[1] for m in metrics]
    var1_values = [m[2] for m in metrics]
    var2_values = [m[3] for m in metrics]
    entropy1_values = [m[4] for m in metrics]
    entropy2_values = [m[5] for m in metrics]
    
    # Plot the metrics
    plt.figure(figsize=(10, 3))
    plt.subplot(1, 2, 1)
    # plt.gca().set_ylim([0, 0.1])
    plt.plot(epochs, mse1_values, 'g', label='Left variation')
    plt.title('Variation')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
    # plt.subplot(3, 2, 2)
    plt.plot(epochs, mse2_values, 'c', label='Right variation')
    # plt.title('Var2')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.gca().set_ylim([0, 0.1])
    plt.plot(epochs, var1_values, 'g', label='Left variation')
    plt.title('Variation')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
    # plt.subplot(3, 2, 4)
    plt.plot(epochs, var2_values, 'c', label='Right variation')
    # plt.title('Var2')
    plt.xlabel('Epochs')
    plt.ylabel('Value')
    plt.legend()
    
#     plt.subplot(3, 2, 5)
#     plt.plot(epochs, entropy1_values, 'm', label='Entropy1')
#     plt.title('Entropy1')
#     plt.xlabel('Epochs')
#     plt.ylabel('Value')
#     plt.legend()
    
#     # plt.subplot(3, 2, 6)
#     plt.plot(epochs, entropy2_values, 'y', label='Entropy2')
#     plt.title('Entropy2')
#     plt.xlabel('Epochs')
#     plt.ylabel('Value')
#     plt.legend()
    plt.savefig('tests/var_bce_plot.png')
#     plt.tight_layout()
#     plt.suptitle(title)
#     plt.show()

# Example usage
plot_metrics(classification_dict, 'Metrics Over Training')


In [ ]:
# classification_dict

In [ ]:
plot_metrics(ss, 'Metrics Over Training')

In [ ]:
import matplotlib.pyplot as plt

# Define a function to plot the metrics
def plot_metrics(metrics: List[Tuple[float, float, float, float, float, float]], title: str):
    epochs = range(1, len(metrics) + 1)
    
    mse1_values = [m[0] for m in metrics]
    mse2_values = [m[1] for m in metrics]
    var1_values = [m[2] for m in metrics]
    var2_values = [m[3] for m in metrics]
    entropy1_values = [m[4] for m in metrics]
    entropy2_values = [m[5] for m in metrics]
    
    # Plot the metrics
    plt.figure(figsize=(12, 8))
    
#     plt.subplot(3, 2, 1)
#     # plt.gca().set_ylim([0, 0.1])
#     plt.plot(epochs, mse1_values, 'b', label='MSE1')
#     plt.title('MSE1')
#     plt.xlabel('Epochs')
#     plt.ylabel('Value')
#     plt.legend()
    
#     # plt.subplot(3, 2, 2)
#     plt.plot(epochs, mse2_values, 'r', label='MSE2')
#     plt.title('MSE2')
#     plt.xlabel('Epochs')
#     plt.ylabel('Value')
#     plt.legend()
    
    plt.subplot(2, 2, 1)
    # plt.gca().set_ylim([0, 0.1])
    plt.plot(epochs, var1_values, 'g', label='Var1')
    # plt.title('Var1')
    plt.xlabel('Epochs')
    plt.ylabel('Variance')
    plt.legend()
    
    # plt.subplot(3, 2, 4)
    plt.plot(epochs, var2_values, 'c', label='Var2')
    # plt.title('Var2')
    plt.xlabel('Epochs')
    plt.ylabel('Variance')
    plt.legend()
    
    plt.subplot(2, 2, 2)
    
    # plt.subplot(1, 2, 1)
    plt.gca().set_ylim([0, 0.1])
    plt.plot(epochs, var1_values, 'g', label='Var1')
    # plt.title('Var1')
    plt.xlabel('Epochs')
    plt.ylabel('Variance')
    plt.legend()
    
    # plt.subplot(3, 2, 4)
    plt.plot(epochs, var2_values, 'c', label='Var2')
    # plt.title('Var2')
    plt.xlabel('Epochs')
    plt.ylabel('Variance')
    plt.legend()
    
#     plt.subplot(3, 2, 5)
#     plt.plot(epochs, entropy1_values, 'm', label='Entropy1')
#     plt.title('Entropy1')
#     plt.xlabel('Epochs')
#     plt.ylabel('Value')
#     plt.legend()
    
#     # plt.subplot(3, 2, 6)
#     plt.plot(epochs, entropy2_values, 'y', label='Entropy2')
#     plt.title('Entropy2')
#     plt.xlabel('Epochs')
#     plt.ylabel('Value')
#     plt.legend()
    plt.suptitle(title)
    plt.tight_layout()
    # plt.suptitle(title)
    plt.savefig('tests/metrics_bce_plot.png')
    plt.show()

# Example usage
plot_metrics(ss, 'Train with standart BCE')
# plot_metrics(s, 'Train with weighted BCE')


In [ ]:
plot_metrics(ss, 'Variances')

In [ ]:
import matplotlib.pyplot as plt

# Define a function to plot the metrics
def plot_metrics(metrics: List[Tuple[float, float, float, float, float, float]],
                 metrics2: List[Tuple[float, float, float, float, float, float]],
                 title: str):
    epochs = range(1, len(metrics) + 1)
    
    mse1_values = [m[0] for m in metrics]
    mse2_values = [m[1] for m in metrics]
    var1_values = [m[2] for m in metrics]
    var2_values = [m[3] for m in metrics]
    entropy1_values = [m[4] for m in metrics]
    entropy2_values = [m[5] for m in metrics]
    
    mse1_values2 = [m[0] for m in metrics2]
    mse2_values2 = [m[1] for m in metrics2]
    var1_values2 = [m[2] for m in metrics2]
    var2_values2 = [m[3] for m in metrics2]
    entropy1_values2 = [m[4] for m in metrics2]
    entropy2_values2 = [m[5] for m in metrics2]
    
    # Plot the metrics
    plt.figure(figsize=(12, 8))
    
#     plt.subplot(3, 2, 1)
#     # plt.gca().set_ylim([0, 0.1])
#     plt.plot(epochs, mse1_values, 'b', label='MSE1')
#     plt.title('MSE1')
#     plt.xlabel('Epochs')
#     plt.ylabel('Value')
#     plt.legend()
    
#     # plt.subplot(3, 2, 2)
#     plt.plot(epochs, mse2_values, 'r', label='MSE2')
#     plt.title('MSE2')
#     plt.xlabel('Epochs')
#     plt.ylabel('Value')
#     plt.legend()
    
    plt.subplot(2, 2, 1)
    # plt.gca().set_ylim([0, 0.001])
    plt.plot(epochs, var1_values, 'g', label='Left variance')
    # plt.title('Var1')
    plt.xlabel('Epochs')
    plt.ylabel('Variance')
    plt.legend()
    
    # plt.subplot(3, 2, 4)
    plt.plot(epochs, var2_values, 'c', label='Right variance')
    # plt.title('Var2')
    plt.xlabel('Epochs')
    plt.ylabel('Variance')
    plt.legend()
    
    plt.subplot(2, 2, 2)
    plt.gca().set_ylim([0, 0.1])
    plt.plot(epochs, var1_values, 'g', label='Left variance')
    # plt.title('Var1')
    plt.xlabel('Epochs')
    plt.ylabel('Variance')
    plt.legend()
    
    # plt.subplot(3, 2, 4)
    plt.plot(epochs, var2_values, 'c', label='Right variance')
    # plt.title('Var2')
    plt.xlabel('Epochs')
    plt.ylabel('Variance')
    plt.legend()
    
    
    plt.subplot(2, 2, 3)
    
    # plt.subplot(1, 2, 1)
    # plt.gca().set_ylim([0, 0.1])
    
    plt.plot(epochs, var1_values2, 'g', label='Left variance')
    # plt.title('Var1')
    plt.xlabel('Epochs')
    plt.ylabel('Variance')
    plt.legend()
    
    # plt.subplot(3, 2, 4)
    plt.plot(epochs, var2_values2, 'c', label='Right variance')
    # plt.title('Var2')
    plt.xlabel('Epochs')
    plt.ylabel('Variance')
    plt.legend()
    
    plt.subplot(2, 2, 4)
    
    # plt.subplot(1, 2, 1)
    plt.gca().set_ylim([0, 0.1])
    
    plt.plot(epochs, var1_values2, 'g', label='Left variance')
    # plt.title('Var1')
    plt.xlabel('Epochs')
    plt.ylabel('Variance')
    plt.legend()
    
    # plt.subplot(3, 2, 4)
    plt.plot(epochs, var2_values2, 'c', label='Right variance')
    # plt.title('Var2')
    plt.xlabel('Epochs')
    plt.ylabel('Variance')
    plt.legend()
    
    # plt.title('Variance')
    
#     plt.subplot(3, 2, 5)
#     plt.plot(epochs, entropy1_values, 'm', label='Entropy1')
#     plt.title('Entropy1')
#     plt.xlabel('Epochs')
#     plt.ylabel('Value')
#     plt.legend()
    
#     # plt.subplot(3, 2, 6)
#     plt.plot(epochs, entropy2_values, 'y', label='Entropy2')
#     plt.title('Entropy2')
#     plt.xlabel('Epochs')
#     plt.ylabel('Value')
#     plt.legend()
    
    plt.tight_layout()
    # plt.suptitle(title)
    # plt.savefig('tests/metrics_bce_wbce_plot.png')
    plt.show()

# Example usage
# plot_metrics(s, 'Variances')

plot_metrics(ss, s, 'Variances')

In [ ]:
import matplotlib.pyplot as plt

# Define a function to plot the metrics
def plot_metrics(metrics: List[Tuple[float, float, float, float, float, float]],
                 metrics2: List[Tuple[float, float, float, float, float, float]],
                 title: str):
    epochs = range(1, len(metrics) + 1)
    
    mse1_values = [m[0] for m in metrics]
    mse2_values = [m[1] for m in metrics]
    var1_values = [m[2] for m in metrics]
    var2_values = [m[3] for m in metrics]
    entropy1_values = [m[4] for m in metrics]
    entropy2_values = [m[5] for m in metrics]
    
    mse1_values2 = [m[0] for m in metrics2]
    mse2_values2 = [m[1] for m in metrics2]
    var1_values2 = [m[2] for m in metrics2]
    var2_values2 = [m[3] for m in metrics2]
    entropy1_values2 = [m[4] for m in metrics2]
    entropy2_values2 = [m[5] for m in metrics2]
    
    # Plot the metrics
    plt.figure(figsize=(12, 8))
    
#     plt.subplot(3, 2, 1)
#     # plt.gca().set_ylim([0, 0.1])
#     plt.plot(epochs, mse1_values, 'b', label='MSE1')
#     plt.title('MSE1')
#     plt.xlabel('Epochs')
#     plt.ylabel('Value')
#     plt.legend()
    
#     # plt.subplot(3, 2, 2)
#     plt.plot(epochs, mse2_values, 'r', label='MSE2')
#     plt.title('MSE2')
#     plt.xlabel('Epochs')
#     plt.ylabel('Value')
#     plt.legend()
    
    plt.subplot(2, 2, 1)
    # plt.gca().set_ylim([0, 0.001])
    plt.plot(epochs, var1_values, 'g', label='Left variance BCE')
    # plt.title('Var1')
    plt.xlabel('Epochs')
    plt.ylabel('Variance')
    plt.legend()
    
    # plt.subplot(3, 2, 4)
    plt.plot(epochs, var1_values2, 'c', label='Left variance wBCE')
    # plt.title('Var2')
    plt.xlabel('Epochs')
    plt.ylabel('Variance')
    plt.legend()
    
    plt.subplot(2, 2, 2)
    # plt.gca().set_ylim([0, 0.1])
    plt.plot(epochs, var2_values, 'g', label='Right variance BCE')
    # plt.title('Var1')
    plt.xlabel('Epochs')
    plt.ylabel('Variance')
    plt.legend()
    
    # plt.subplot(3, 2, 4)
    plt.plot(epochs, var2_values2, 'c', label='Right variance wBCE')
    # plt.title('Var2')
    plt.xlabel('Epochs')
    plt.ylabel('Variance')
    plt.legend()
    
    
    plt.subplot(2, 2, 3)
    plt.gca().set_ylim([0, 0.001])
    plt.plot(epochs, var1_values, 'g', label='Left variance BCE')
    # plt.title('Var1')
    plt.xlabel('Epochs')
    plt.ylabel('Variance')
    plt.legend()
    
    # plt.subplot(3, 2, 4)
    plt.plot(epochs, var1_values2, 'c', label='Left variance wBCE')
    # plt.title('Var2')
    plt.xlabel('Epochs')
    plt.ylabel('Variance')
    plt.legend()
    
    plt.subplot(2, 2, 4)
    plt.gca().set_ylim([0, 0.1])
    plt.plot(epochs, var2_values, 'g', label='Right variance BCE')
    # plt.title('Var1')
    plt.xlabel('Epochs')
    plt.ylabel('Variance')
    plt.legend()
    
    # plt.subplot(3, 2, 4)
    plt.plot(epochs, var2_values2, 'c', label='Right variance wBCE')
    # plt.title('Var2')
    plt.xlabel('Epochs')
    plt.ylabel('Variance')
    plt.legend()
    
    # plt.title('Variance')
    
#     plt.subplot(3, 2, 5)
#     plt.plot(epochs, entropy1_values, 'm', label='Entropy1')
#     plt.title('Entropy1')
#     plt.xlabel('Epochs')
#     plt.ylabel('Value')
#     plt.legend()
    
#     # plt.subplot(3, 2, 6)
#     plt.plot(epochs, entropy2_values, 'y', label='Entropy2')
#     plt.title('Entropy2')
#     plt.xlabel('Epochs')
#     plt.ylabel('Value')
#     plt.legend()
    
    plt.tight_layout()
    # plt.suptitle(title)
    plt.savefig('tests/metrics1_bce_wbce_plot.png')
    plt.show()

# Example usage
# plot_metrics(s, 'Variances')

plot_metrics(ss, s, 'Variances')

In [ ]:
pwd

In [ ]:
with open('/Users/serafim/Desktop/Job/projects/science/hse/GAN-Estimation-Uncertainty/uncertainty/tests/base_gan__variance__10_split__accurate_wieights_0,01_OneCycleLR__lr_0,0001__1/metrics.pkl', 'wb') as f:
    pickle.dump(s, f)

In [ ]:
1/2 * np.log(1/2) + 1/2 * np.log(1/2)

In [ ]:
3/4 * np.log(3/4) + 1/4 * np.log(1/4)

In [ ]:
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import OneCycleLR

lr = 0.0001
NUM_EPOCHS = 5000
# loss_function = WeightedIntervalCrossEntropyLoss()
loss_function = WeightedVarianceCrossEntropyLoss()
save_path = '/Users/serafim/Desktop/Job/projects/science/hse/GAN-Estimation-Uncertainty/uncertainty/tests_range_5_eWBCELoss/'
if not os.path.exists(save_path):
    os.makedirs(save_path)
for i in range(26):
    D = Base_Discriminator().to(device=device)
    G = Base_Generator().to(device=device)

    D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
    G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
    
    NAME = f'base_gan__variance__each_point__accurate_wieights_0,01_OneCycleLR__lr_0,0001__{i}'
    # Adjust these parameters as needed
    max_lr = 0.0005  # Maximum learning rate

    # Define the scheduler for both D_optimizer and G_optimizer
    scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
    scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)


    D_losses_final, G_losses_final, Variances, Weights_variance, s = train(
        num_epochs = NUM_EPOCHS,
        data_loader = train_loader,
        D = D,
        G = G,
        D_optimizer = D_optimizer,
        G_optimizer = G_optimizer, 
        criterion = loss_function,
        device = device,
        plot_process = True,
        save_path = save_path,
        name = f'{NAME}',
        weights_interval = True,
        # plot_info = True,
        animate_bar_var = True,
        progress_generator = True,
        n_split = 10,
        scheduler_D = scheduler_D,
        scheduler_G = scheduler_G,
        info_n = NUM_EPOCHS,
        save_mse = 'G_mse.pkl'
    )
# plot_sine(G, save_path = save_path, name = NAME)


In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 5000
loss_function = WeightedIntervalCrossEntropyLoss()
# loss_function = nn.BCELoss()

D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
# NAME = 'base_gan__lr_0,001_OneCycleLR__1_1'
NAME = 'base_gan__variance__10_split__accurate_wieights_0,01_OneCycleLR__lr_0,0001__5_1'
# Adjust these parameters as needed
max_lr = 0.0005  # Maximum learning rate

# Define the scheduler for both D_optimizer and G_optimizer
scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)


D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = True,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    n_split = 10,
    scheduler_D = scheduler_D,
    scheduler_G = scheduler_G,
    info_n = 200
)
# plot_sine(G, save_path = save_path, name = NAME)


In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 5000
loss_function = WeightedIntervalCrossEntropyLoss()
# loss_function = nn.BCELoss()

D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
# NAME = 'base_gan__lr_0,001_OneCycleLR__1_1'
NAME = 'base_gan__variance__10_split__accurate_wieights_0,01_OneCycleLR__lr_0,0001__6_1'
# Adjust these parameters as needed
max_lr = 0.0005  # Maximum learning rate

# Define the scheduler for both D_optimizer and G_optimizer
scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)


D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = True,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    n_split = 10,
    scheduler_D = scheduler_D,
    scheduler_G = scheduler_G,
    info_n = 200
)
# plot_sine(G, save_path = save_path, name = NAME)


In [ ]:

# D = Base_Discriminator().to(device=device)
# G = Base_Generator().to(device=device)
# lr = 0.0001
# NUM_EPOCHS = 5000
# loss_function = WeightedIntervalCrossEntropyLoss()
# # loss_function = nn.BCELoss()

# D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
# G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# # NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# # NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
# # NAME = 'base_gan__lr_0,001_OneCycleLR__1_1'
# NAME = 'base_gan__variance__10_split__accurate_wieights_0,01_OneCycleLR__lr_0,0001__2_1'
# # Adjust these parameters as needed
# max_lr = 0.0005  # Maximum learning rate

# # Define the scheduler for both D_optimizer and G_optimizer
# scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
# scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)


# D_losses_final, G_losses_final, Variances, Weights_variance = train(
#     num_epochs = NUM_EPOCHS,
#     data_loader = train_loader,
#     D = D,
#     G = G,
#     D_optimizer = D_optimizer,
#     G_optimizer = G_optimizer, 
#     criterion = loss_function,
#     device = device,
#     plot_process = True,
#     save_path = save_path,
#     name = f'{NAME}',
#     weights_interval = True,
#     # plot_info = True,
#     animate_bar_var = True,
#     progress_generator = True,
#     n_split = 10,
#     scheduler_D = scheduler_D,
#     scheduler_G = scheduler_G
# )
# # plot_sine(G, save_path = save_path, name = NAME)


In [ ]:
lr = 0.0001
NUM_EPOCHS = 5000
loss_function = nn.BCELoss()
max_lr = 0.0005
save_path = '/Users/serafim/Desktop/Job/projects/science/hse/GAN-Estimation-Uncertainty/uncertainty/tests_range_2_BCELoss/'
if not os.path.exists(save_path):
    os.makedirs(save_path)
    
for i in range(50):
    D = Base_Discriminator().to(device=device)
    G = Base_Generator().to(device=device)

    D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
    G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
    NAME = f'base_gan__lr_0,001_OneCycleLR__{i}'

# Adjust these parameters as needed


    scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
    scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)

    D_losses_final, G_losses_final, Variances, Weights_variance = train(
        num_epochs = NUM_EPOCHS,
        data_loader = train_loader,
        D = D,
        G = G,
        D_optimizer = D_optimizer,
        G_optimizer = G_optimizer, 
        criterion = loss_function,
        device = device,
        plot_process = True,
        save_path = save_path,
        name = f'{NAME}',
        weights_interval = False,
        # plot_info = True,
        animate_bar_var = True,
        progress_generator = True,
        scheduler_D = scheduler_D,
        scheduler_G = scheduler_G,
        info_n = NUM_EPOCHS,
        save_mse = 'G_mse.pkl'
        # n_split = 10
    )

In [ ]:
import pickle
wbce_mse_path = 'tests_range_4_WBCELoss/G_mse.pkl'

with open(wbce_mse_path, 'rb') as f:
    loaded_wbce_G_mse = pickle.load(f)

In [ ]:
import pickle
ewbce_mse_path = 'tests_range_5_eWBCELoss/G_mse.pkl'

with open(ewbce_mse_path, 'rb') as f:
    loaded_ewbce_G_mse = pickle.load(f)


In [ ]:
bce_mse_path = 'tests_range_2_BCELoss/G_mse.pkl'

with open(bce_mse_path, 'rb') as f:
    loaded_bce_G_mse = pickle.load(f)

In [ ]:
# save_path
import pickle
import matplotlib.pyplot as plt
import numpy as np

def plot_mse_from_pickle_bar(file_path1: str, file_path2: str) -> None:
    """
    Load and plot MSE values from a pickle file as bar plots.
    
    Parameters:
    - file_path: Path to the pickle file containing MSE values.
    """
    # Load the MSE values from the pickle file
    with open(file_path1, 'rb') as f:
        loaded_bce_G_mse = pickle.load(f)
    with open(file_path2, 'rb') as f:
        loaded_wbce_G_mse = pickle.load(f)
    if len(loaded_bce_G_mse) != len(loaded_wbce_G_mse):
        min_len = min(len(loaded_bce_G_mse), len(loaded_wbce_G_mse))
        
        loaded_bce_G_mse = loaded_bce_G_mse[:min_len]
        loaded_wbce_G_mse = loaded_wbce_G_mse[:min_len]
        print(len(loaded_bce_G_mse), len(loaded_wbce_G_mse))
    
    # assert(len(loaded_bce_G_mse) == len(loaded_wbce_G_mse))
    # Extract MSE values from loaded_G_mse
    mse1_values_1 = [mse[0] for mse in loaded_bce_G_mse]
    mse2_values_1 = [mse[1] for mse in loaded_bce_G_mse]
    
    mse1_values_2 = [mse[0] for mse in loaded_wbce_G_mse]
    mse2_values_2 = [mse[1] for mse in loaded_wbce_G_mse]

    # Calculate average MSE values
    avg_mse1_1 = sum(mse1_values_1) / len(mse1_values_1)
    avg_mse2_1 = sum(mse2_values_1) / len(mse2_values_1)

    avg_mse1_2 = sum(mse1_values_2) / len(mse1_values_2)
    avg_mse2_2 = sum(mse2_values_2) / len(mse2_values_2)
    

    # Plot MSE values as bar plots
    fig, ax = plt.subplots(figsize=(10, 6))
    bar_width = 0.35
    index = np.arange(len(mse1_values_1))
    
    
    rects1 = ax.bar(index, mse1_values_1, bar_width, label='MSE for left part of sine for bce GAN')
    rects2 = ax.bar(index + bar_width, mse1_values_2, bar_width, label='MSE for left part of sine for wbce GAN')
    # rects2 = ax.bar(index + bar_width, mse2_values, bar_width, label='MSE for right part of sine')

    ax.set_xlabel('Iterations')
    ax.set_ylabel('MSE')
    ax.set_title('MSE over iterations')
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels([str(i) for i in range(len(mse1_values_1))])
    ax.legend()
    
    # Add text for average MSE values
    ax.text(0.5, 0.9, f'Average MSE for left part bce: {avg_mse1_1:.4f}', transform=ax.transAxes, fontsize=10, verticalalignment='top')
    ax.text(0.5, 0.85, f'Average MSE for left part wbce: {avg_mse1_2:.4f}', transform=ax.transAxes, fontsize=10, verticalalignment='top')
    # ax.text(0.5, 0.85, f'Average MSE for right part: {avg_mse2:.4f}', transform=ax.transAxes, fontsize=10, verticalalignment='top')

    
    
    rects1 = ax.bar(index, mse1_values_1, bar_width, label='MSE for right part of sine for bce GAN')
    rects2 = ax.bar(index + bar_width, mse1_values_2, bar_width, label='MSE for right part of sine for wbce GAN')
    # rects2 = ax.bar(index + bar_width, mse2_values, bar_width, label='MSE for right part of sine')

    ax.set_xlabel('Iterations')
    ax.set_ylabel('MSE')
    ax.set_title('MSE over iterations')
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels([str(i) for i in range(len(mse2_values_1))])
    ax.legend()
    
    # Add text for average MSE values
    ax.text(0.5, 0.9, f'Average MSE for right part bce: {avg_mse2_1:.4f}', transform=ax.transAxes, fontsize=10, verticalalignment='top')
    ax.text(0.5, 0.85, f'Average MSE for right part wbce: {avg_mse2_2:.4f}', transform=ax.transAxes, fontsize=10, verticalalignment='top')
    
    
    plt.show()

# Example usage:
plot_mse_from_pickle_bar(bce_mse_path, ewbce_mse_path)


In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np

def plot_mse_from_pickle_bar(file_path1: str, file_path2: str) -> None:
    """
    Load and plot MSE values from a pickle file as bar plots.
    
    Parameters:
    - file_path: Path to the pickle file containing MSE values.
    """
    # Load the MSE values from the pickle file
    with open(file_path1, 'rb') as f:
        loaded_bce_G_mse = pickle.load(f)
    with open(file_path2, 'rb') as f:
        loaded_wbce_G_mse = pickle.load(f)
    assert(len(loaded_bce_G_mse) == len(loaded_wbce_G_mse))
    # Extract MSE values from loaded_G_mse
    mse1_values_1 = [mse[0] for mse in loaded_bce_G_mse]
    mse2_values_1 = [mse[1] for mse in loaded_bce_G_mse]
    
    mse1_values_2 = [mse[0] for mse in loaded_wbce_G_mse]
    mse2_values_2 = [mse[1] for mse in loaded_wbce_G_mse]

    # Calculate average MSE values
    avg_mse1_1 = sum(mse1_values_1) / len(mse1_values_1)
    avg_mse2_1 = sum(mse2_values_1) / len(mse2_values_1)

    avg_mse1_2 = sum(mse1_values_2) / len(mse1_values_2)
    avg_mse2_2 = sum(mse2_values_2) / len(mse2_values_2)
    

    # Plot MSE values as bar plots
    fig, ax = plt.subplots(figsize=(10, 6))
    bar_width = 0.35
    index = np.arange(len(mse1_values_1))
    
    
    rects1 = ax.bar(index, mse1_values_1, bar_width, label='MSE for left part of sine for bce GAN')
    rects2 = ax.bar(index + bar_width, mse1_values_2, bar_width, label='MSE for left part of sine for wbce GAN')
    # rects2 = ax.bar(index + bar_width, mse2_values, bar_width, label='MSE for right part of sine')

    ax.set_xlabel('Iterations')
    ax.set_ylabel('MSE')
    ax.set_title('MSE over iterations')
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels([str(i) for i in range(len(mse1_values_1))])
    ax.legend()
    
    # Add text for average MSE values
    ax.text(0.5, 0.9, f'Average MSE for left part bce: {avg_mse1_1:.4f}', transform=ax.transAxes, fontsize=10, verticalalignment='top')
    ax.text(0.5, 0.85, f'Average MSE for left part wbce: {avg_mse1_2:.4f}', transform=ax.transAxes, fontsize=10, verticalalignment='top')
    # ax.text(0.5, 0.85, f'Average MSE for right part: {avg_mse2:.4f}', transform=ax.transAxes, fontsize=10, verticalalignment='top')

    
    
    rects1 = ax.bar(index, mse1_values_1, bar_width, label='MSE for right part of sine for bce GAN')
    rects2 = ax.bar(index + bar_width, mse1_values_2, bar_width, label='MSE for right part of sine for wbce GAN')
    # rects2 = ax.bar(index + bar_width, mse2_values, bar_width, label='MSE for right part of sine')

    ax.set_xlabel('Iterations')
    ax.set_ylabel('MSE')
    ax.set_title('MSE over iterations')
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels([str(i) for i in range(len(mse2_values_1))])
    ax.legend()
    
    # Add text for average MSE values
    ax.text(0.5, 0.9, f'Average MSE for right part bce: {avg_mse2_1:.4f}', transform=ax.transAxes, fontsize=10, verticalalignment='top')
    ax.text(0.5, 0.85, f'Average MSE for right part wbce: {avg_mse2_2:.4f}', transform=ax.transAxes, fontsize=10, verticalalignment='top')
    
    
    plt.show()

# Example usage:
plot_mse_from_pickle_bar(bce_mse_path, wbce_mse_path)


In [ ]:

        
import pickle
import matplotlib.pyplot as plt
import numpy as np

def plot_mse_from_pickle_bar(file_path1: str, file_path2: str) -> None:
    """
    Load and plot MSE values from two pickle files as bar plots.
    
    Parameters:
    - file_path1: Path to the first pickle file containing MSE values.
    - file_path2: Path to the second pickle file containing MSE values.
    """
    # Load the MSE values from the first pickle file
    with open(file_path1, 'rb') as f:
        loaded_bce_G_mse = pickle.load(f)
        
    # Load the MSE values from the second pickle file
    with open(file_path2, 'rb') as f:
        loaded_wbce_G_mse = pickle.load(f)
        
    if len(loaded_bce_G_mse) != len(loaded_wbce_G_mse):
        min_len = min(len(loaded_bce_G_mse), len(loaded_wbce_G_mse))
        
        loaded_bce_G_mse = loaded_bce_G_mse[:min_len]
        loaded_wbce_G_mse = loaded_wbce_G_mse[:min_len]
        print(len(loaded_bce_G_mse), len(loaded_wbce_G_mse))
    # assert len(loaded_bce_G_mse) == len(loaded_wbce_G_mse)
    
    # Extract MSE values for the left and right parts of the sine function
    mse1_values_bce = [mse[0] for mse in loaded_bce_G_mse]
    mse2_values_bce = [mse[1] for mse in loaded_bce_G_mse]
    
    mse1_values_wbce = [mse[0] for mse in loaded_wbce_G_mse]
    mse2_values_wbce = [mse[1] for mse in loaded_wbce_G_mse]

    # Calculate average and median MSE values for both parts
    avg_mse1_bce = np.mean(mse1_values_bce)
    avg_mse2_bce = np.mean(mse2_values_bce)
    median_mse1_bce = np.median(mse1_values_bce)
    median_mse2_bce = np.median(mse2_values_bce)
    
    avg_mse1_wbce = np.mean(mse1_values_wbce)
    avg_mse2_wbce = np.mean(mse2_values_wbce)
    median_mse1_wbce = np.median(mse1_values_wbce)
    median_mse2_wbce = np.median(mse2_values_wbce)

    # Plot MSE values as bar plots
    fig, axes = plt.subplots(2, figsize=(10, 12))
    index = np.arange(len(mse1_values_bce))
    bar_width = 0.35

    # Plot MSE values for the left part of the sine function
    axes[0].bar(index, mse1_values_bce, bar_width, label='MSE for left part of sine for BCE GAN')
    axes[0].bar(index + bar_width, mse1_values_wbce, bar_width, label='MSE for left part of sine for WBCE GAN')
    axes[0].set_xlabel('Iterations')
    axes[0].set_ylabel('MSE')
    axes[0].set_title('MSE for Left Part of Sine')
    axes[0].set_xticks(index + bar_width / 2)
    axes[0].set_xticklabels([str(i) for i in range(len(mse1_values_bce))])
    axes[0].legend()
    print(f'The left part:\n Mean MSE: BCE GAN -- {avg_mse1_bce:.5f}, WBCE GAN -- {avg_mse1_wbce:.5f},\n Median MSE: BCE GAN -- {median_mse1_bce:.5f}, WBCE GAN -- {median_mse1_wbce:.5f}')
    # axes[0].text(0.5, 0.9, f'Average MSE for left part BCE: {avg_mse1_bce:.4f}', transform=axes[0].transAxes, fontsize=10, verticalalignment='top')
    # axes[0].text(0.5, 0.85, f'Median MSE for left part BCE: {median_mse1_bce:.4f}', transform=axes[0].transAxes, fontsize=10, verticalalignment='top')
    # axes[0].text(0.5, 0.8, f'Average MSE for left part WBCE: {avg_mse1_wbce:.4f}', transform=axes[0].transAxes, fontsize=10, verticalalignment='top')
    # axes[0].text(0.5, 0.75, f'Median MSE for left part WBCE: {median_mse1_wbce:.4f}', transform=axes[0].transAxes, fontsize=10, verticalalignment='top')

    # Plot MSE values for the right part of the sine function
    axes[1].bar(index, mse2_values_bce, bar_width, label='MSE for right part of sine for BCE GAN')
    axes[1].bar(index + bar_width, mse2_values_wbce, bar_width, label='MSE for right part of sine for WBCE GAN')
    axes[1].set_xlabel('Iterations')
    axes[1].set_ylabel('MSE')
    axes[1].set_title('MSE for Right Part of Sine')
    axes[1].set_xticks(index + bar_width / 2)
    axes[1].set_xticklabels([str(i) for i in range(len(mse2_values_bce))])
    axes[1].legend()
    # axes[1].text(0.5, 0.9, f'Average MSE for right part BCE: {avg_mse2_bce:.4f}', transform=axes[1].transAxes, fontsize=10, verticalalignment='top')
    # axes[1].text(0.5, 0.85, f'Median MSE for right part BCE: {median_mse2_bce:.4f}', transform=axes[1].transAxes, fontsize=10, verticalalignment='top')
    # axes[1].text(0.5, 0.8, f'Average MSE for right part WBCE: {avg_mse2_wbce:.4f}', transform=axes[1].transAxes, fontsize=10, verticalalignment='top')
    # axes[1].text(0.5, 0.75, f'Median MSE for right part WBCE: {median_mse2_wbce:.4f}', transform=axes[1].transAxes, fontsize=10, verticalalignment='top')
    print(f'\nThe right part:\n Mean MSE: BCE GAN -- {avg_mse2_bce:.5f}, WBCE GAN -- {avg_mse2_wbce:.5f},\n Median MSE: BCE GAN -- {median_mse2_bce:.5f}, WBCE GAN -- {median_mse2_wbce:.5f}')
    plt.tight_layout()
    plt.show()


plot_mse_from_pickle_bar(bce_mse_path, ewbce_mse_path)


In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np

def plot_mse_from_pickle_bar(file_path1: str, file_path2: str) -> None:
    """
    Load and plot MSE values from two pickle files as bar plots.
    
    Parameters:
    - file_path1: Path to the first pickle file containing MSE values.
    - file_path2: Path to the second pickle file containing MSE values.
    """
    # Load the MSE values from the first pickle file
    with open(file_path1, 'rb') as f:
        loaded_bce_G_mse = pickle.load(f)
        
    # Load the MSE values from the second pickle file
    with open(file_path2, 'rb') as f:
        loaded_wbce_G_mse = pickle.load(f)
        
    assert len(loaded_bce_G_mse) == len(loaded_wbce_G_mse)
    
    # Extract MSE values for the left and right parts of the sine function
    mse1_values_bce = [mse[0] for mse in loaded_bce_G_mse]
    mse2_values_bce = [mse[1] for mse in loaded_bce_G_mse]
    
    mse1_values_wbce = [mse[0] for mse in loaded_wbce_G_mse]
    mse2_values_wbce = [mse[1] for mse in loaded_wbce_G_mse]

    # Calculate average and median MSE values for both parts
    avg_mse1_bce = np.mean(mse1_values_bce)
    avg_mse2_bce = np.mean(mse2_values_bce)
    median_mse1_bce = np.median(mse1_values_bce)
    median_mse2_bce = np.median(mse2_values_bce)
    
    avg_mse1_wbce = np.mean(mse1_values_wbce)
    avg_mse2_wbce = np.mean(mse2_values_wbce)
    median_mse1_wbce = np.median(mse1_values_wbce)
    median_mse2_wbce = np.median(mse2_values_wbce)

    # Plot MSE values as bar plots
    fig, axes = plt.subplots(2, figsize=(10, 12))
    index = np.arange(len(mse1_values_bce))
    bar_width = 0.35

    # Plot MSE values for the left part of the sine function
    axes[0].bar(index, mse1_values_bce, bar_width, label='MSE for left part of sine for BCE GAN')
    axes[0].bar(index + bar_width, mse1_values_wbce, bar_width, label='MSE for left part of sine for WBCE GAN')
    axes[0].set_xlabel('Iterations')
    axes[0].set_ylabel('MSE')
    axes[0].set_title('MSE for Left Part of Sine')
    axes[0].set_xticks(index + bar_width / 2)
    axes[0].set_xticklabels([str(i) for i in range(len(mse1_values_bce))])
    axes[0].legend()
    print(f'The left part:\n Mean MSE: BCE GAN -- {avg_mse1_bce:.5f}, WBCE GAN -- {avg_mse1_wbce:.5f},\n Median MSE: BCE GAN -- {median_mse1_bce:.5f}, WBCE GAN -- {median_mse1_wbce:.5f}')
    # axes[0].text(0.5, 0.9, f'Average MSE for left part BCE: {avg_mse1_bce:.4f}', transform=axes[0].transAxes, fontsize=10, verticalalignment='top')
    # axes[0].text(0.5, 0.85, f'Median MSE for left part BCE: {median_mse1_bce:.4f}', transform=axes[0].transAxes, fontsize=10, verticalalignment='top')
    # axes[0].text(0.5, 0.8, f'Average MSE for left part WBCE: {avg_mse1_wbce:.4f}', transform=axes[0].transAxes, fontsize=10, verticalalignment='top')
    # axes[0].text(0.5, 0.75, f'Median MSE for left part WBCE: {median_mse1_wbce:.4f}', transform=axes[0].transAxes, fontsize=10, verticalalignment='top')

    # Plot MSE values for the right part of the sine function
    axes[1].bar(index, mse2_values_bce, bar_width, label='MSE for right part of sine for BCE GAN')
    axes[1].bar(index + bar_width, mse2_values_wbce, bar_width, label='MSE for right part of sine for WBCE GAN')
    axes[1].set_xlabel('Iterations')
    axes[1].set_ylabel('MSE')
    axes[1].set_title('MSE for Right Part of Sine')
    axes[1].set_xticks(index + bar_width / 2)
    axes[1].set_xticklabels([str(i) for i in range(len(mse2_values_bce))])
    axes[1].legend()
    # axes[1].text(0.5, 0.9, f'Average MSE for right part BCE: {avg_mse2_bce:.4f}', transform=axes[1].transAxes, fontsize=10, verticalalignment='top')
    # axes[1].text(0.5, 0.85, f'Median MSE for right part BCE: {median_mse2_bce:.4f}', transform=axes[1].transAxes, fontsize=10, verticalalignment='top')
    # axes[1].text(0.5, 0.8, f'Average MSE for right part WBCE: {avg_mse2_wbce:.4f}', transform=axes[1].transAxes, fontsize=10, verticalalignment='top')
    # axes[1].text(0.5, 0.75, f'Median MSE for right part WBCE: {median_mse2_wbce:.4f}', transform=axes[1].transAxes, fontsize=10, verticalalignment='top')
    print(f'\nThe right part:\n Mean MSE: BCE GAN -- {avg_mse2_bce:.5f}, WBCE GAN -- {avg_mse2_wbce:.5f},\n Median MSE: BCE GAN -- {median_mse2_bce:.5f}, WBCE GAN -- {median_mse2_wbce:.5f}')
    plt.tight_layout()
    plt.show()


plot_mse_from_pickle_bar(bce_mse_path, wbce_mse_path)


In [ ]:
import sklearn
sklearn.__version__

In [ ]:
sklearn==1.2.2


In [ ]:
import pickle
import matplotlib.pyplot as plt

# file_path = 'G_mse.pkl'
# file_path = os.path.join(save_path, file_path)

bce_mse_path = 'tests_range_2_BCELoss/G_mse.pkl'
wbce_mse_path = 'tests_range_4_WBCELoss/G_mse.pkl'

def plot_mse_from_pickle(file_path: str) -> None:
    """
    Load and plot MSE values from a pickle file.
    
    Parameters:
    - file_path: Path to the pickle file containing MSE values.
    """
    # Load the MSE values from the pickle file
    with open(file_path, 'rb') as f:
        loaded_G_mse = pickle.load(f)
    # print(loaded_G_mse)

    # Extract MSE values from loaded_G_mse
    mse1_values = [mse[0] for mse in loaded_G_mse]
    mse2_values = [mse[1] for mse in loaded_G_mse]

    # Calculate average MSE values
    avg_mse1 = sum(mse1_values) / len(mse1_values)
    avg_mse2 = sum(mse2_values) / len(mse2_values)

    # Plot MSE values
    plt.figure(figsize=(10, 6))
    plt.plot(mse1_values, label='MSE for left part of sine')
    plt.plot(mse2_values, label='MSE for right part of sine')
    plt.xlabel('Iterations')
    plt.ylabel('MSE')
    plt.title('MSE over iterations')
    plt.legend()
    plt.grid(True)
    
    # Add text for average MSE values
    plt.text(0.5, 0.9, f'Average MSE for left part: {avg_mse1:.4f}', transform=plt.gca().transAxes, fontsize=10, verticalalignment='top')
    plt.text(0.5, 0.85, f'Average MSE for right part: {avg_mse2:.4f}', transform=plt.gca().transAxes, fontsize=10, verticalalignment='top')
    
    plt.show()

# Example usage:
plot_mse_from_pickle(wbce_mse_path)


In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np

def plot_mse_from_pickle_bar(file_path: str) -> None:
    """
    Load and plot MSE values from a pickle file as bar plots.
    
    Parameters:
    - file_path: Path to the pickle file containing MSE values.
    """
    # Load the MSE values from the pickle file
    with open(file_path, 'rb') as f:
        loaded_G_mse = pickle.load(f)

    # Extract MSE values from loaded_G_mse
    mse1_values = [mse[0] for mse in loaded_G_mse]
    mse2_values = [mse[1] for mse in loaded_G_mse]

    # Calculate average MSE values
    avg_mse1 = sum(mse1_values) / len(mse1_values)
    avg_mse2 = sum(mse2_values) / len(mse2_values)

    # Plot MSE values as bar plots
    fig, ax = plt.subplots(figsize=(10, 6))
    bar_width = 0.35
    index = np.arange(len(mse1_values))
    rects1 = ax.bar(index, mse1_values, bar_width, label='MSE for left part of sine')
    rects2 = ax.bar(index + bar_width, mse2_values, bar_width, label='MSE for right part of sine')

    ax.set_xlabel('Iterations')
    ax.set_ylabel('MSE')
    ax.set_title('MSE over iterations')
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels([str(i) for i in range(len(mse1_values))])
    ax.legend()
    
    # Add text for average MSE values
    ax.text(0.5, 0.9, f'Average MSE for left part: {avg_mse1:.4f}', transform=ax.transAxes, fontsize=10, verticalalignment='top')
    ax.text(0.5, 0.85, f'Average MSE for right part: {avg_mse2:.4f}', transform=ax.transAxes, fontsize=10, verticalalignment='top')

    plt.show()

# Example usage:
plot_mse_from_pickle_bar(file_path)


In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np

def plot_mse_from_pickle_bar(file_path: str) -> None:
    """
    Load and plot MSE values from a pickle file as bar plots.
    
    Parameters:
    - file_path: Path to the pickle file containing MSE values.
    """
    # Load the MSE values from the pickle file
    with open(file_path, 'rb') as f:
        loaded_G_mse = pickle.load(f)

    # Extract MSE values from loaded_G_mse
    mse1_values = [mse[0] for mse in loaded_G_mse]
    mse2_values = [mse[1] for mse in loaded_G_mse]

    # Calculate average MSE values
    avg_mse1 = sum(mse1_values) / len(mse1_values)
    avg_mse2 = sum(mse2_values) / len(mse2_values)

    # Plot MSE values as bar plots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12))
    
    index = np.arange(len(mse1_values))
    bar_width = 0.35

    rects1 = ax1.bar(index, mse1_values, bar_width, label='MSE for left part of sine', color='b')
    ax1.set_ylabel('MSE')
    ax1.set_title('MSE for left part of sine')
    ax1.set_xticks(index)
    ax1.set_xticklabels([str(i) for i in range(len(mse1_values))])
    ax1.legend()
    
    rects2 = ax2.bar(index, mse2_values, bar_width, label='MSE for right part of sine', color='r')
    ax2.set_ylabel('MSE')
    ax2.set_title('MSE for right part of sine')
    ax2.set_xticks(index)
    ax2.set_xticklabels([str(i) for i in range(len(mse2_values))])
    ax2.legend()

    # Add text for average MSE values
    ax1.text(0.5, 0.9, f'Average MSE for left part: {avg_mse1:.4f}', transform=ax1.transAxes, fontsize=10, verticalalignment='top', color='b')
    ax2.text(0.5, 0.9, f'Average MSE for right part: {avg_mse2:.4f}', transform=ax2.transAxes, fontsize=10, verticalalignment='top', color='r')

    plt.tight_layout()
    plt.show()

# Example usage:
plot_mse_from_pickle_bar(bce_mse_path)


In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np

def plot_mse_from_pickle_bar(file_path: str) -> None:
    """
    Load and plot MSE values from a pickle file as bar plots, with rects1 on the left and rects2 on the right.
    
    Parameters:
    - file_path: Path to the pickle file containing MSE values.
    """
    # Load the MSE values from the pickle file
    with open(file_path, 'rb') as f:
        loaded_G_mse = pickle.load(f)

    # Extract MSE values from loaded_G_mse
    mse1_values = [mse[0] for mse in loaded_G_mse]
    mse2_values = [mse[1] for mse in loaded_G_mse]

    # Calculate average MSE values
    avg_mse1 = sum(mse1_values) / len(mse1_values)
    avg_mse2 = sum(mse2_values) / len(mse2_values)

    # Plot MSE values as bar plots with rects1 on the left and rects2 on the right
    fig, ax = plt.subplots(figsize=(10, 6))
    bar_width = 0.35
    index = np.arange(len(mse1_values))
    rects1 = ax.bar(index, mse1_values, bar_width, label='MSE for left part of sine', align='edge')
    rects2 = ax.bar(index + bar_width, mse2_values, bar_width, label='MSE for right part of sine', align='edge')

    ax.set_xlabel('Iterations')
    ax.set_ylabel('MSE')
    ax.set_title('MSE over iterations')
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels([str(i) for i in range(len(mse1_values))])
    ax.legend()
    
    # Add text for average MSE values
    ax.text(0.5, 0.9, f'Average MSE for left part: {avg_mse1:.4f}', transform=ax.transAxes, fontsize=10, verticalalignment='top')
    ax.text(0.5, 0.85, f'Average MSE for right part: {avg_mse2:.4f}', transform=ax.transAxes, fontsize=10, verticalalignment='top')

    plt.show()

# Example usage:
plot_mse_from_pickle_bar(file_path)


In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np

def plot_mse_distribution(file_path: str) -> None:
    """
    Load and plot the distribution of MSE values from a pickle file as histograms, 
    with rects1 on the left and rects2 on the right.
    
    Parameters:
    - file_path: Path to the pickle file containing MSE values.
    """
    # Load the MSE values from the pickle file
    with open(file_path, 'rb') as f:
        loaded_G_mse = pickle.load(f)

    # Extract MSE values from loaded_G_mse
    mse1_values = [mse[0] for mse in loaded_G_mse]
    mse2_values = [mse[1] for mse in loaded_G_mse]

    # Calculate average MSE values
    avg_mse1 = sum(mse1_values) / len(mse1_values)
    avg_mse2 = sum(mse2_values) / len(mse2_values)

    # Plot MSE distributions as histograms with rects1 on the left and rects2 on the right
    fig, axs = plt.subplots(1, 2, figsize=(12, 6), sharey=True)
    
    axs[0].hist(mse1_values, bins=2, color='blue', alpha=0.7, label='MSE for left part of sine')
    axs[1].hist(mse2_values, bins=2, color='orange', alpha=0.7, label='MSE for right part of sine')

    axs[0].set_xlabel('MSE')
    axs[0].set_ylabel('Frequency')
    axs[0].set_title('Distribution of MSE for left part')
    axs[1].set_xlabel('MSE')
    axs[1].set_title('Distribution of MSE for right part')
    
    # Add text for average MSE values
    axs[0].text(0.5, 0.9, f'Average MSE: {avg_mse1:.4f}', transform=axs[0].transAxes, fontsize=10, verticalalignment='top')
    axs[1].text(0.5, 0.9, f'Average MSE: {avg_mse2:.4f}', transform=axs[1].transAxes, fontsize=10, verticalalignment='top')
    
    plt.tight_layout()
    plt.show()

# Example usage:
plot_mse_distribution(file_path)


In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 5000
# loss_function = WeightedIntervalCrossEntropyLoss()
loss_function = nn.BCELoss()
D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
NAME = 'base_gan__lr_0,001_OneCycleLR__8_1'

# Adjust these parameters as needed
max_lr = 0.0005

scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)

D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = False,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    scheduler_D = scheduler_D,
    scheduler_G = scheduler_G,
    info_n = 200
    # n_split = 10
)

In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 5000
# loss_function = WeightedIntervalCrossEntropyLoss()
loss_function = nn.BCELoss()
D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
NAME = 'base_gan__lr_0,001_OneCycleLR__15_1'

# Adjust these parameters as needed
max_lr = 0.0005

scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)

D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = False,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    scheduler_D = scheduler_D,
    scheduler_G = scheduler_G,
    info_n = 200
    # n_split = 10
)

In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 5000
# loss_function = WeightedIntervalCrossEntropyLoss()
loss_function = nn.BCELoss()
D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
NAME = 'base_gan__lr_0,001_OneCycleLR__16_1'

# Adjust these parameters as needed
max_lr = 0.0005

scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)

D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = False,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    scheduler_D = scheduler_D,
    scheduler_G = scheduler_G,
    info_n = 200
    # n_split = 10
)

In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 5000
# loss_function = WeightedIntervalCrossEntropyLoss()
loss_function = nn.BCELoss()
D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
NAME = 'base_gan__lr_0,001_OneCycleLR__17_1'

# Adjust these parameters as needed
max_lr = 0.0005

scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)

D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = False,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    scheduler_D = scheduler_D,
    scheduler_G = scheduler_G,
    info_n = 200
    # n_split = 10
)

In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 5000
# loss_function = WeightedIntervalCrossEntropyLoss()
loss_function = nn.BCELoss()
D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
NAME = 'base_gan__lr_0,001_OneCycleLR__18_1'

# Adjust these parameters as needed
max_lr = 0.0005

scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)

D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = False,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    scheduler_D = scheduler_D,
    scheduler_G = scheduler_G,
    info_n = 200
    # n_split = 10
)

In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 5000
# loss_function = WeightedIntervalCrossEntropyLoss()
loss_function = nn.BCELoss()
D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
NAME = 'base_gan__lr_0,001_OneCycleLR__9_1'

# Adjust these parameters as needed
max_lr = 0.0005

scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)

D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = False,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    scheduler_D = scheduler_D,
    scheduler_G = scheduler_G,
    info_n = 200
    # n_split = 10
)

In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 5000
# loss_function = WeightedIntervalCrossEntropyLoss()
loss_function = nn.BCELoss()
D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
NAME = 'base_gan__lr_0,001_OneCycleLR__10_1'

# Adjust these parameters as needed
max_lr = 0.0005

scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)

D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = False,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    scheduler_D = scheduler_D,
    scheduler_G = scheduler_G,
    info_n = 200
    # n_split = 10
)

In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 5000
# loss_function = WeightedIntervalCrossEntropyLoss()
loss_function = nn.BCELoss()
D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
NAME = 'base_gan__lr_0,001_OneCycleLR__11_1'

# Adjust these parameters as needed
max_lr = 0.0005

scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)

D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = False,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    scheduler_D = scheduler_D,
    scheduler_G = scheduler_G,
    info_n = 200
    # n_split = 10
)

In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 5000
loss_function = WeightedIntervalCrossEntropyLoss()
# loss_function = nn.BCELoss()

D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
# NAME = 'base_gan__lr_0,001_OneCycleLR__1_1'
NAME = 'base_gan__variance__2_split__accurate_wieights_0,01_OneCycleLR__lr_0,0001__12_1'
# Adjust these parameters as needed
max_lr = 0.0005  # Maximum learning rate

# Define the scheduler for both D_optimizer and G_optimizer
scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)


D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = True,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    n_split = 2,
    scheduler_D = scheduler_D,
    scheduler_G = scheduler_G,
    info_n = 200
)
# plot_sine(G, save_path = save_path, name = NAME)


In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 5000
loss_function = WeightedIntervalCrossEntropyLoss()
# loss_function = nn.BCELoss()

D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
# NAME = 'base_gan__lr_0,001_OneCycleLR__1_1'
NAME = 'base_gan__variance__6_split__accurate_wieights_0,01_OneCycleLR__lr_0,0001__13_1'
# Adjust these parameters as needed
max_lr = 0.0005  # Maximum learning rate

# Define the scheduler for both D_optimizer and G_optimizer
scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)


D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = True,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    n_split = 6,
    scheduler_D = scheduler_D,
    scheduler_G = scheduler_G,
    info_n = 200
)
# plot_sine(G, save_path = save_path, name = NAME)


In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 5000
loss_function = WeightedIntervalCrossEntropyLoss()
# loss_function = nn.BCELoss()

D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
# NAME = 'base_gan__lr_0,001_OneCycleLR__1_1'
NAME = 'base_gan__variance__20_split__accurate_wieights_0,01_OneCycleLR__lr_0,0001__14_1'
# Adjust these parameters as needed
max_lr = 0.0005  # Maximum learning rate

# Define the scheduler for both D_optimizer and G_optimizer
scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)


D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = True,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    n_split = 20,
    scheduler_D = scheduler_D,
    scheduler_G = scheduler_G,
    info_n = 200
)
# plot_sine(G, save_path = save_path, name = NAME)


In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 1000
loss_function = WeightedIntervalCrossEntropyLoss()
# loss_function = nn.BCELoss()

D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
# NAME = 'base_gan__lr_0,001_OneCycleLR__1_1'
NAME = 'base_gan__variance__10_split__accurate_wieights_0,01_OneCycleLR__lr_0,001__3_1'
# Adjust these parameters as needed
max_lr = 0.0005  # Maximum learning rate

# Define the scheduler for both D_optimizer and G_optimizer
scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)


D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = True,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    n_split = 4,
    scheduler_D = scheduler_D,
    scheduler_G = scheduler_G
)
# plot_sine(G, save_path = save_path, name = NAME)


In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 1000
loss_function = WeightedIntervalCrossEntropyLoss()
# loss_function = nn.BCELoss()

D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
# NAME = 'base_gan__lr_0,001_OneCycleLR__1_1'
NAME = 'base_gan__variance_std__2_split__accurate_wieights_0,01_OneCycleLR__lr_0,001__4_1'
# Adjust these parameters as needed
max_lr = 0.0005  # Maximum learning rate

# Define the scheduler for both D_optimizer and G_optimizer
scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)


D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = True,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    n_split = 2,
    scheduler_D = scheduler_D,
    scheduler_G = scheduler_G
)
# plot_sine(G, save_path = save_path, name = NAME)


In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 1000
loss_function = WeightedIntervalCrossEntropyLoss()
# loss_function = nn.BCELoss()

D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
# NAME = 'base_gan__lr_0,001_OneCycleLR__1_1'
NAME = 'base_gan__variance_std__25_split__accurate_wieights_0,01_OneCycleLR__lr_0,001__5_1'
# Adjust these parameters as needed
max_lr = 0.0005  # Maximum learning rate

# Define the scheduler for both D_optimizer and G_optimizer
scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)


D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = True,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    n_split = 25,
    scheduler_D = scheduler_D,
    scheduler_G = scheduler_G
)
# plot_sine(G, save_path = save_path, name = NAME)


In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 5000
loss_function = WeightedIntervalCrossEntropyLoss()
# loss_function = nn.BCELoss()

D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
# NAME = 'base_gan__lr_0,001_OneCycleLR__1_1'
NAME = 'base_gan__variance__10_split__accurate_wieights_0,01_OneCycleLR__lr_0,001__3_1'
# Adjust these parameters as needed
max_lr = 0.0005  # Maximum learning rate

# Define the scheduler for both D_optimizer and G_optimizer
scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)


D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = True,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    n_split = 10,
    scheduler_D = scheduler_D,
    scheduler_G = scheduler_G
)
# plot_sine(G, save_path = save_path, name = NAME)


In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 1000
# loss_function = WeightedIntervalCrossEntropyLoss()
loss_function = nn.BCELoss()
D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
NAME = 'base_gan__lr_0,001_OneCycleLR__6_1'

# Adjust these parameters as needed
max_lr = 0.001  # Maximum learning rate

# Define the scheduler for both D_optimizer and G_optimizer
scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)

D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = False,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    scheduler_D = scheduler_D,
    scheduler_G = scheduler_G
    # n_split = 10
)

In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 1000
loss_function = WeightedIntervalCrossEntropyLoss()
# loss_function = nn.BCELoss()

D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
# NAME = 'base_gan__lr_0,001_OneCycleLR__1_1'
NAME = 'base_gan__variance_10_split__accurate_wieights_0,01_OneCycleLR__lr_0,001__3_1'
# Adjust these parameters as needed
max_lr = 0.001  # Maximum learning rate

# Define the scheduler for both D_optimizer and G_optimizer
scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)

D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = True,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    # scheduler_D = scheduler_D,
    # scheduler_G = scheduler_G
    n_split = 10
)

In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 1000
# loss_function = WeightedIntervalCrossEntropyLoss()
loss_function = nn.BCELoss()
D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
NAME = 'base_gan__part_dataset__lr_0,001_OneCycleLR__10_1'

# Adjust these parameters as needed
max_lr = 0.001  # Maximum learning rate

# Define the scheduler for both D_optimizer and G_optimizer
scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)

D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = True,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    n_split = 10,
    scheduler_D = scheduler_D,
    scheduler_G = scheduler_G
)

In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 1000
# loss_function = WeightedIntervalCrossEntropyLoss()
loss_function = nn.BCELoss()
D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
NAME = 'base_gan__lr_0,001_OneCycleLR__6_1'

# Adjust these parameters as needed
max_lr = 0.001  # Maximum learning rate

# Define the scheduler for both D_optimizer and G_optimizer
scheduler_D = OneCycleLR(D_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)
scheduler_G = OneCycleLR(G_optimizer, max_lr=max_lr, total_steps=NUM_EPOCHS * NUM_BATCHES)

D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = False,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    # n_split = 10
)

In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 1000
loss_function = WeightedIntervalCrossEntropyLoss()
D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
NAME = 'base_gan__variance_10_split__accurate_wieights_0,01__lr_0,0001__3_1'
D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = True,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    n_split = 10
)

In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.0001
NUM_EPOCHS = 1000
loss_function = WeightedIntervalCrossEntropyLoss()
D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
# NAME = 'base_gan__variance_2_split__accurate_wieights__4_1'
# NAME = 'base_gan__variance_10_split__accurate_wieights__lr_0,0001__11_1'
NAME = 'base_gan__variance_10_split__accurate_wieights_0,01__lr_0,0001__4_1'
D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = True,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    n_split = 10
)

In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.001
NUM_EPOCHS = 1000
loss_function = WeightedIntervalCrossEntropyLoss()
D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
NAME = 'base_gan__variance_20_split__accurate_wieights__5_1'

D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = True,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    n_split = 20
    
)

In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.001
NUM_EPOCHS = 1000
loss_function = WeightedIntervalCrossEntropyLoss()
D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
NAME = 'base_gan__variance_50_split__accurate_wieights__6_1'

D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = True,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    n_split = 50
    
)

In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.001
NUM_EPOCHS = 1000
loss_function = WeightedIntervalCrossEntropyLoss()
D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
NAME = 'base_gan__variance_100_split__accurate_wieights__7_1'

D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = True,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    n_split = 100
    
)

In [ ]:

D = Base_Discriminator().to(device=device)
G = Base_Generator().to(device=device)
lr = 0.001
NUM_EPOCHS = 1000
loss_function = WeightedIntervalCrossEntropyLoss()
D_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr)
NAME = 'base_gan__variance_200_split__accurate_wieights__8_1'

D_losses_final, G_losses_final, Variances, Weights_variance = train(
    num_epochs = NUM_EPOCHS,
    data_loader = train_loader,
    D = D,
    G = G,
    D_optimizer = D_optimizer,
    G_optimizer = G_optimizer, 
    criterion = loss_function,
    device = device,
    plot_process = True,
    save_path = save_path,
    name = f'{NAME}',
    weights_interval = True,
    # plot_info = True,
    animate_bar_var = True,
    progress_generator = True,
    n_split = 200
    
)

In [ ]:
for i in range(5):
    print(i)

In [ ]:

def create_gif(file_paths, gif_path, save_path, name, duration=NUM_EPOCHS, loop=0):
    images = [plt.imread(file_path) for file_path in file_paths]
    
    plt.figure(figsize=(12, 6))
    plt.axis('off')
    ims = plt.imshow(images[0])

    def update(i):
        ims.set_array(images[i])

    ani = FuncAnimation(plt.gcf(), update, frames=len(images), interval=duration, repeat_delay=1000)
    
    gif_path = os.path.join(save_path, NAME, 'final_generated_plots.gif')
    ani.save(gif_path, writer='imagemagick', fps=2, dpi=80)
    
    # Delete all files except the last epoch
    for file_path in file_paths[:-1]:
        os.remove(file_path)

In [ ]:
# file_paths

In [ ]:
file_paths = [os.path.join(save_path, NAME, f'generated_plots_epoch_{epoch}.png') for epoch in range(0, 40, 20)]
create_gif(file_paths, save_path = save_path, name = NAME, gif_path='final_generated_plots.gif')

In [ ]:
# Weights_variance

In [ ]:
# NUM_EXPERIMENTS = 10
# NUM_EPOCHS = 20

# # Initialize lists to store results for each experiment
# all_D_losses_final = []
# all_G_losses_final = []
# all_Variances = []

# for experiment in range(NUM_EXPERIMENTS):
#     # Initialize your models, optimizers, etc. here if needed
    
#     # Run the training process
#     D_losses_final, G_losses_final, Variances = train(
#         num_epochs=NUM_EPOCHS,
#         data_loader=train_loader,
#         D=D,
#         G=G,
#         D_optimizer=D_optimizer,
#         G_optimizer=G_optimizer,
#         criterion=loss_function,
#         device=device,
#         plot_process=False,  # Disable plotting for individual experiments
#         save_path=save_path,
#         name=f'{NAME}_experiment_{experiment}',
#         weights_interval=True
#     )
    
#     plot_sine(G, save_path = save_path, name = NAME)
    
#     # Store results for this experiment
#     all_D_losses_final.append(D_losses_final)
#     all_G_losses_final.append(G_losses_final)
#     all_Variances.append(Variances)

# # After all experiments, you can analyze the aggregated results or perform any desired post-processing.

# # For example, you can print the average and standard deviation of losses across experiments:
# avg_D_losses_final = sum(all_D_losses_final) / NUM_EXPERIMENTS
# avg_G_losses_final = sum(all_G_losses_final) / NUM_EXPERIMENTS
# avg_Variances = sum(all_Variances) / NUM_EXPERIMENTS

# print(f'Average D_losses_final: {avg_D_losses_final}')
# print(f'Average G_losses_final: {avg_G_losses_final}')
# print(f'Average Variances: {avg_Variances}')


In [ ]:
plot_sine(G, save_path = None)

In [ ]:
# [el[0] for el in vals]
['Category 1', 'Category 2'], [lst1[0], lst2[0]]

In [ ]:
# cats
# [el[0] for el in vals]

In [ ]:
cats, vals = zip(*Weights_variance.items())


In [ ]:
# vals
# cats

In [ ]:
from modules.animate import animated_bar_var_plot

In [ ]:
animated_bar_var_plot(Weights_variance, filename = save_path)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
cats, vals = zip(*Weights_variance.items())
fig, axes = plt.subplots(figsize=(8, 6))
axes.set_ylim(0, 1)

bars = axes.bar(cats, [el[0] for el in vals])
cats, vals = zip(*Weights_variance.items())
def animate(i):
    for j in range(len(cats)):
        bars[j].set_height(vals[j][i])

plt.title("Intro to Animated Bar Plot", fontsize=14)
ani = FuncAnimation(fig, animate, frames=len(vals[0]), repeat=False)
HTML(ani.to_jshtml())




In [ ]:
from typing import Dict, Callable, Tuple, List
def covert_dict2animate_format(weights_variance: dict) -> List[List]:
    names = weights_variance.keys()
    values = weights_variance.values()
    return list(names), list(values)
    

In [ ]:
# covert_dict2animate_format(Weights_variance)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

fig, axes = plt.subplots(figsize=(8, 6))
axes.set_ylim(0, 120)

lst1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
lst2 = [0, 5, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80, 90, 100]

bars = axes.bar(['Category 1', 'Category 2'], [lst1[0], lst2[0]])

def animate(i):
    bars[0].set_height(lst1[i])
    bars[1].set_height(lst2[i])

plt.title("Intro to Animated Bar Plot", fontsize=14)
ani = FuncAnimation(fig, animate, frames=len(lst1), repeat=False)
HTML(ani.to_jshtml())


In [ ]:
q1, q2 = weights_variances(G)
q1, q2

In [ ]:
r1, r2 = weights_variances(G)
r1, r2

In [ ]:
t1 = {v: r1[k] for k, v in r2.items()}
t2 = {v: q1[k] for k, v in q2.items()}

In [ ]:
t1, t2

In [ ]:
q1.values(), q2.values()

In [ ]:
t1

In [ ]:
plt.bar(q2.values(), q1.values())

In [ ]:
q1.keys()

In [ ]:
q1.values()

In [ ]:
# pip install PyQt5


In [ ]:
# pip install PySide2


In [ ]:
from IPython.display import HTML
HTML(ani.to_jshtml())

In [ ]:
ax.set_ylim(0, max(max(dic.values()) for dic in q))

In [ ]:
from matplotlib import animation
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import seaborn as sns
%matplotlib qt

In [ ]:
%matplotlib qt
fig = plt.figure(figsize=(8,6))
axes = fig.add_subplot(1,1,1)
axes.set_ylim(0, 150)
plt.style.use("seaborn")

lst1=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15 ]
lst2=[0, 5, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80, 90, 100]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

q = [t1, t2]
i = 0

fig, ax = plt.subplots()
xdata, ydata = [], []
ln, = ax.plot([], [], 'ro')

def init():
    ax.set_xlim(0, 2*np.pi)
    ax.set_ylim(-1, 1)
    return ln,

# def update(frame):
    
#     xdata.append(frame)
#     ydata.append(np.sin(frame))
#     ln.set_data(xdata, ydata)
#     return ln,
# x_labels = list(q[0].keys())
def update(frame):
    heights = [q[i][key] for key in x_labels]
    for bar, height in zip(bars, heights):
        bar.set_height(height)
    return bars

ani = FuncAnimation(fig, update, frames=np.linspace(0, 2*np.pi, 128),
                    init_func=init, blit=True, repeat = False)

HTML(ani.to_jshtml())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Sample data for q
q = [{'A': 3, 'B': 5, 'C': 2}, {'A': 1, 'B': 4, 'C': 6}, {'A': 2, 'B': 3, 'C': 5}]

fig, ax = plt.subplots()
xdata, ydata = [], []
bars = ax.bar([], [])

def init():
    ax.set_ylim(0, max(max(dic.values()) for dic in q))
    return bars

def update(frame):
    values = q[frame].values()
    xdata.extend(range(len(values)))
    ydata.extend(values)
    
    for bar, h in zip(bars, values):
        bar.set_height(h)
    
    return bars

ani = FuncAnimation(fig, update, frames=range(len(q)),
                    init_func=init, blit=True, repeat=False)

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots()
xdata, ydata = [], []
ln, = ax.plot([], [], 'ro')

def init():
    ax.set_xlim(0, 2*np.pi)
    ax.set_ylim(-1, 1)
    return ln,

def update(frame):
    print(frame, np.sin(frame))
    xdata.append(frame)
    ydata.append(np.sin(frame))
    ln.set_data(xdata, ydata)
    return ln,

ani = FuncAnimation(fig, update, frames=np.linspace(0, 2*np.pi, 128),
                    init_func=init, blit=True, repeat = False)
# plt.show();

In [ ]:
update

In [ ]:
# D_losses_final = []
# G_losses_final = []
# Variances = []
# # NUM_EPOCHS = NUM_EPOCHS//10
# for epoch in tqdm(range(1, NUM_EPOCHS+1)):           
#     D_losses, G_losses = [], []
#     for batch_idx in range(batches_per_epoch):
#         batch, mean_var = create_batch(
#                                 Generator = G,
#                                 batch_size = BATCH_SIZE,
#                                 get_variance = True,
#                                 repeat = 10,
#                                 num_samples = 128
#                             )
#         x, info = batch#x is y coordiante, we predict it, info is x corrdinate, its condition
#         D_losses.append(D_train(x, info))
#         G_losses.append(G_train(x))
#         Variances.append(mean_var)
#         # print(len(batch), var, x.size(), info.size())
#         # print(var)
#         # break
    
#     # for batch_idx, (x, info) in enumerate(train_loader):
#     #     D_losses.append(D_train(x, info))
#     #     G_losses.append(G_train(x))
        
#     D_losses_final.append(torch.mean(torch.FloatTensor(D_losses)))
#     G_losses_final.append(torch.mean(torch.FloatTensor(G_losses)))
    
#     if epoch % 20 == 0:
#         print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (
#                 (epoch),
#                 NUM_EPOCHS,
#                 torch.mean(torch.FloatTensor(D_losses)),
#                 torch.mean(torch.FloatTensor(G_losses))
#                 )
#              )

In [ ]:
q1, q2 = weights_variances(G)

# q


In [ ]:
# weights_variancies

In [ ]:
q2

In [ ]:
q1

In [ ]:
# q

In [ ]:
# q
a = torch.zeros(32, 1)
a.squeeze(-1).size()

In [ ]:
    # calculate binary cross-entropy loss
    loss = - (y_true * torch.log(y_pred) + (1 - y_true) * torch.log(1 - y_pred))

    # apply weights if provided
    if weights is not None:
        loss = loss * weights

    # take the mean across the batch
    loss = torch.mean(loss)
    
    loss = F.binary_cross_entropy_with_logits(y_pred, y_true, weight=weights)

The structure of having an outer function (weighted_interval_crossentropy) that returns an inner function (loss) is a way to create a closure, allowing the inner function to access variables from the outer function

In [ ]:
import torch
import torch.nn.functional as F

def weighted_interval_crossentropy():
    """
    A weighted version of interval crossentropy loss for PyTorch.

    weights: torch tensor of shape batch_size
             torch.tensor([0.1, 0.3, 0.2,...]) # point one is in interval where the weight is 0.1, point 2 in interval the weight is 0.3, ...
    """
    # weights = torch.tensor(weights, requires_grad=False)
    

    def loss(y_pred, y_true, weights, conditional_info, from_logits=False):
        """
        y_true size: torch tensor (32, 1)
        y_pred size: torch tensor (32, 1)
        """
        weights = weights4batch(weights[0],
                                conditional_info = conditional_info,
                                bins_values = weights[1])# weights size: torch tensor (32)
        if from_logits:
            y_pred = F.softmax(y_pred, dim=-1)

        # clip to prevent NaN's and Inf's
        y_pred = torch.clamp(y_pred, 1e-7, 1 - 1e-7)
        
        loss = F.binary_cross_entropy(y_pred, y_true, weight=weights.unsqueeze(-1))

        # # calculate loss
        # weights = weights.unsqueeze(-1)
        # print(f'''weights: {weights}, 
        #       y_true: {y_true},
        #       y_pred: {y_pred},
        #       torch.log(y_pred): {torch.log(y_pred)}''')
        # loss = y_true * torch.log(y_pred) * weights * 10
        # loss = -torch.sum(loss.squeeze(-1), -1)
        # print(f'loss: {loss}')
        return loss
    
    return loss


# Example usage:
# weights = [0.5, 2, 10]
# loss_function = weighted_categorical_crossentropy(weights)
# loss_value = loss_function(y_true, y_pred, )


In [ ]:
# y_pred

In [ ]:
torch.clamp(torch.tensor([0, 0.5, 1, 2, 3]), 1e-7, 1 - 1e-7)

In [ ]:
torch.tensor([1, 2, 3]), torch.log(torch.tensor([0, 0.5, 1, 2, 3]))


In [ ]:
# weights = [0.5, 2, 10]
loss_function = weighted_categorical_crossentropy(weights)
loss_value = loss_function(y_true, y_pred, )

In [ ]:
q1

In [ ]:
# def cross_

In [ ]:
weighted_interval_crossentropy()

In [ ]:
weighted_interval_crossentropy()

In [ ]:
q1

In [ ]:
def weights4batch(weights:dict, conditional_info, bins_values:dict):
    
    """
    Assign weights for every point based on conditional information and bins.

    Args:
    - weights (dict): Weights for every interval.
    - conditional_info (torch.Tensor): Tensor with conditional information (coordinate x of sine).
    - bins_values (dict): Bins values.

    Returns:
    - torch.tensor: Weights for every point that we have in condition.
    """
    weights_bins = []
    number_bins = len(bins_values.values())
    # print(f'conditional_info: {conditional_info}')
    sorted_info, indices = torch.sort(conditional_info, axis = 0)

    
    current_bin_id = 1
    # print(f'sorted_info and indices: {sorted_info}, {indices}')
    for i in range(conditional_info.size(0)):
        #find the next current_bin_id
        while current_bin_id <= number_bins - 1 and not (
            bins_values[current_bin_id] < sorted_info[i].item() < bins_values[current_bin_id + 1]
        ):
            current_bin_id += 1
            
        weights_bins.append(weights[current_bin_id])
        
    weights_bins = torch.tensor(weights_bins)
    sorted_weights = weights_bins[torch.argsort(indices.view(-1))]
    # print(torch.argsort(indices.view(-1)))
    return sorted_weights


                
            
            
            

        
        
    
    

In [ ]:
conditional_info_0 = 2 * math.pi * torch.rand(4).view(-1, 1).to(device)
# conditional_info_0

In [ ]:
q1, q2
# conditional_info_0
# q2

In [ ]:
q2

In [ ]:
conditional_info_0

In [ ]:
e = weights4batch(q1, conditional_info_0, q2)
# e[1]
e

In [ ]:
e.size()

In [ ]:
# e[1][e[0]]
# e[1]
# [e[0]]

In [ ]:
# indices
# bi

In [ ]:
r = torch.log(torch.tensor([torch.tensor(i) for i in range(1, 33)]))

In [ ]:
sorted, indices = torch.sort(fi, axis = 0)

In [ ]:
e.size(), r.size()

In [ ]:
# r
e.view(-1)

In [ ]:
e

In [ ]:
r * e
# y_true * torch.log(y_pred) * weights

In [ ]:
fi = 2 * math.pi * torch.rand(32).view(-1, 1).to(device)

In [ ]:
# fi

In [ ]:
plot_sine(G, save_path = '/Users/serafim/Desktop/Job/projects/science/hse/GAN-Estimation-Uncertainty/uncertainty/tests/', name = 'base_gan__base_training__1')


In [ ]:
pwd

## Test

In [ ]:
# latent_space_samples = torch.randn(1000, 2)
num_samples = 10000
latent_space_samples = torch.randn(num_samples, 1)
# info = 8 * math.pi * torch.rand(num_samples).view(-1, 1) - 4 * math.pi
info = 2 * math.pi * torch.rand(num_samples).view(-1, 1)
generated_samples = G(latent_space_samples, info)



In [ ]:
points_x, res = calculate_variance(G, repeat = 10, num_samples = 10000)

##### 4
> num_samples = 128, lr = 0.0005

In [ ]:
generated_samples = generated_samples.detach()
plt.plot(info, generated_samples, 'ko', markersize = 0.5)

In [ ]:
# Plot the graph
plt.plot(points_x, res, 'ko', markersize = 3, label='Variancies')
plt.xlabel('x')
plt.ylabel('Model variance')
plt.title('Model variancies at Different Points')
plt.legend()
plt.grid(True)
plt.show()

##### 3
> num_samples = 128

In [ ]:
generated_samples = generated_samples.detach()
plt.plot(info, generated_samples, 'ko', markersize = 0.5)

In [ ]:
# Plot the graph
plt.plot(points_x, res, 'ko', markersize = 3, label='Variancies')
plt.xlabel('x')
plt.ylabel('Model variance')
plt.title('Model variancies at Different Points')
plt.legend()
plt.grid(True)
plt.show()

##### 2

In [ ]:
generated_samples = generated_samples.detach()
plt.plot(info, generated_samples, 'ko', markersize = 0.5)

In [ ]:
# Plot the graph
plt.plot(points_x, res, 'ko', markersize = 3, label='Variancies')
plt.xlabel('x')
plt.ylabel('Model variance')
plt.title('Model variancies at Different Points')
plt.legend()
plt.grid(True)
plt.show()

##### 1

In [ ]:
generated_samples = generated_samples.detach()
plt.plot(info, generated_samples, 'ko', markersize = 0.5)

In [ ]:
# Plot the graph
plt.plot(points_x, res, 'ko', markersize = 3, label='Variancies')
plt.xlabel('x')
plt.ylabel('Model variance')
plt.title('Model variancies at Different Points')
plt.legend()
plt.grid(True)
plt.show()

#### basic

In [ ]:
generated_samples = generated_samples.detach()
plt.plot(info, generated_samples, 'ko', markersize = 0.5)

In [ ]:
points_x, res = calculate_variance(G, repeat = 1000, num_samples = 10000)

In [ ]:
# Plot the graph
plt.plot(points_x, res, 'ko', markersize = 3, label='Variancies')
plt.xlabel('x')
plt.ylabel('Model variance')
plt.title('Model variancies at Different Points')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
create_batch(G, batch_size = 10,                  
                 get_variance = True,
                 repeat = 10,
                 num_samples = 64)